# Chapter 4: Authorization
## Controlling What Users Can Do

### Course: TCPRG4005 - Secure Programming

---

## Learning Objectives

1. **Understand** the difference between authentication and authorization
2. **Implement** secure authorization systems
3. **Avoid** trusting user-supplied authorization data
4. **Apply** principle of least privilege
5. **Design** secure privilege escalation patterns

---

## Key Principle

> **Authorization determines what a user can do AFTER they've been authenticated**

Never trust user-provided privilege information - always check against your own system.

In [ ]:
# Setup for authorization examples
import json
import time
from datetime import datetime
from enum import Enum
from functools import wraps

print("🔐 Chapter 4: Authorization")
print("Controlling user privileges and access")
print("=" * 45)

# Example 1: Vulnerable Authorization - Trusting User Input

The most common authorization vulnerability: trusting data from the user.

In [ ]:
# Demonstrate the classic "?admin=1" vulnerability
class VulnerableSystem:
    def __init__(self):
        self.users = {
            'alice': {'password': 'secret123', 'is_admin': False},
            'bob': {'password': 'password', 'is_admin': False},
            'admin': {'password': 'supersecret', 'is_admin': True}
        }
    
    def vulnerable_check_admin(self, username, user_provided_admin_flag):
        """VULNERABLE: Trusts user-provided admin flag"""
        # BAD: This trusts data from the user
        if user_provided_admin_flag == "1" or user_provided_admin_flag == True:
            print(f"🚨 DANGER: Granting admin access to {username} based on user input!")
            return True
        return False
    
    def secure_check_admin(self, username):
        """SECURE: Checks against internal data only"""
        if username in self.users:
            is_admin = self.users[username]['is_admin']
            print(f"✅ Checking admin status for {username}: {is_admin}")
            return is_admin
        return False

# Demonstrate the vulnerability
system = VulnerableSystem()

print("Authorization Vulnerability Demonstration")
print("-" * 45)

# Normal user tries to access admin function
print("1. Normal access attempt:")
alice_is_admin = system.secure_check_admin('alice')
print(f"   Alice admin status: {alice_is_admin}\n")

# Attacker exploits vulnerable function
print("2. Attack using user-provided admin flag:")
print("   URL: /admin?user=alice&admin=1")
attack_result = system.vulnerable_check_admin('alice', "1")
print(f"   Attack successful: {attack_result}")
print("   🚨 Alice gained admin access without being admin!\n")

# Secure function prevents attack
print("3. Same attack on secure function:")
secure_result = system.secure_check_admin('alice')
print(f"   Attack failed: {secure_result}")
print("   ✅ Secure function only trusts internal data")

# Example 2: Session-Based Authorization

Proper authorization should be tied to authenticated sessions, not user input.

In [ ]:
# Secure session-based authorization system
class SecureAuthorizationSystem:
    def __init__(self):
        self.users = {
            'alice': {'password': 'secret123', 'roles': ['user', 'editor']},
            'bob': {'password': 'password', 'roles': ['user']},
            'admin': {'password': 'supersecret', 'roles': ['user', 'admin', 'editor']}
        }
        self.active_sessions = {}
        self.role_permissions = {
            'user': ['read_posts', 'comment'],
            'editor': ['read_posts', 'comment', 'create_posts', 'edit_posts'],
            'admin': ['read_posts', 'comment', 'create_posts', 'edit_posts', 'delete_posts', 'manage_users']
        }
    
    def login(self, username, password):
        """Authenticate user and create session"""
        if username in self.users and self.users[username]['password'] == password:
            session_id = f"session_{username}_{int(time.time())}"
            self.active_sessions[session_id] = {
                'username': username,
                'roles': self.users[username]['roles'],
                'login_time': datetime.now()
            }
            return session_id
        return None
    
    def has_permission(self, session_id, required_permission):
        """Check if session has required permission"""
        if session_id not in self.active_sessions:
            print(f"❌ No active session found")
            return False
        
        session = self.active_sessions[session_id]
        username = session['username']
        user_roles = session['roles']
        
        # Check if any of the user's roles grant the required permission
        for role in user_roles:
            if role in self.role_permissions:
                if required_permission in self.role_permissions[role]:
                    print(f"✅ {username} has permission '{required_permission}' via role '{role}'")
                    return True
        
        print(f"❌ {username} lacks permission '{required_permission}'")
        return False
    
    def admin_operation(self, session_id, operation):
        """Perform admin-only operation"""
        if not self.has_permission(session_id, 'manage_users'):
            return "Access denied: Admin privileges required"
        
        return f"Admin operation '{operation}' completed successfully"

# Demonstrate secure authorization
auth_system = SecureAuthorizationSystem()

print("\nSecure Session-Based Authorization")
print("-" * 40)

# Alice logs in (regular user)
alice_session = auth_system.login('alice', 'secret123')
print(f"Alice logged in: {alice_session}\n")

# Alice tries to perform operations
print("Alice's permissions:")
auth_system.has_permission(alice_session, 'read_posts')
auth_system.has_permission(alice_session, 'create_posts') 
auth_system.has_permission(alice_session, 'manage_users')

print("\nAlice tries admin operation:")
result = auth_system.admin_operation(alice_session, "delete_user")
print(f"Result: {result}\n")

# Admin logs in
admin_session = auth_system.login('admin', 'supersecret')
print(f"Admin logged in: {admin_session}")

print("\nAdmin performs same operation:")
result = auth_system.admin_operation(admin_session, "delete_user")
print(f"Result: {result}")

# Example 3: Authorization Decorators

Use decorators to enforce authorization requirements consistently.

In [ ]:
# Authorization decorator pattern
def requires_permission(required_permission):
    """Decorator to enforce permission requirements"""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Extract session_id from arguments
            session_id = kwargs.get('session_id') or (args[0] if args else None)
            
            if not session_id:
                return "Error: No session provided"
            
            # Check permission using our auth system
            if not auth_system.has_permission(session_id, required_permission):
                return f"Access denied: '{required_permission}' permission required"
            
            # Permission granted, execute function
            return func(*args, **kwargs)
        return wrapper
    return decorator

def requires_role(required_role):
    """Decorator to enforce role requirements"""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            session_id = kwargs.get('session_id') or (args[0] if args else None)
            
            if not session_id or session_id not in auth_system.active_sessions:
                return "Error: Invalid session"
            
            session = auth_system.active_sessions[session_id]
            if required_role not in session['roles']:
                return f"Access denied: '{required_role}' role required"
            
            return func(*args, **kwargs)
        return wrapper
    return decorator

# Examples using decorators
@requires_permission('create_posts')
def create_blog_post(session_id, title, content):
    """Create a new blog post"""
    session = auth_system.active_sessions[session_id]
    username = session['username']
    return f"Blog post '{title}' created by {username}"

@requires_permission('delete_posts')
def delete_blog_post(session_id, post_id):
    """Delete a blog post"""
    session = auth_system.active_sessions[session_id]
    username = session['username']
    return f"Blog post {post_id} deleted by {username}"

@requires_role('admin')
def system_backup(session_id):
    """Perform system backup"""
    return "System backup completed successfully"

# Test authorization decorators
print("\nAuthorization Decorators")
print("-" * 25)

# Alice (editor) creates a post
print("Alice creates a blog post:")
result = create_blog_post(alice_session, "My First Post", "Hello world!")
print(f"Result: {result}\n")

# Alice tries to delete a post (needs admin permission)
print("Alice tries to delete a post:")
result = delete_blog_post(alice_session, "post123")
print(f"Result: {result}\n")

# Admin deletes a post
print("Admin deletes a post:")
result = delete_blog_post(admin_session, "post123")
print(f"Result: {result}\n")

# Bob (basic user) tries to create a post
bob_session = auth_system.login('bob', 'password')
print("Bob tries to create a post:")
result = create_blog_post(bob_session, "Bob's Post", "Hello!")
print(f"Result: {result}")

# Example 4: Principle of Least Privilege

Grant minimal permissions needed for each operation, use privilege escalation when needed.

In [ ]:
# Principle of least privilege implementation
class PrivilegeEscalationSystem:
    def __init__(self):
        self.temp_privileges = {}  # Temporary elevated privileges
        
    def request_privilege_escalation(self, session_id, required_privilege, justification):
        """Request temporary privilege escalation"""
        if session_id not in auth_system.active_sessions:
            return False, "Invalid session"
        
        session = auth_system.active_sessions[session_id]
        username = session['username']
        
        # Simple approval logic (in real systems, this might require another admin)
        if 'admin' in session['roles'] or 'editor' in session['roles']:
            # Grant temporary privilege for 5 minutes
            expiry = time.time() + 300  # 5 minutes
            
            if session_id not in self.temp_privileges:
                self.temp_privileges[session_id] = {}
            
            self.temp_privileges[session_id][required_privilege] = {
                'granted_at': time.time(),
                'expires_at': expiry,
                'justification': justification,
                'used': False
            }
            
            print(f"✅ Temporary privilege '{required_privilege}' granted to {username}")
            print(f"   Justification: {justification}")
            print(f"   Expires in 5 minutes")
            return True, "Privilege escalation approved"
        
        print(f"❌ Privilege escalation denied for {username}")
        return False, "Insufficient base privileges for escalation"
    
    def has_temp_privilege(self, session_id, privilege):
        """Check if session has temporary privilege"""
        if session_id not in self.temp_privileges:
            return False
        
        if privilege not in self.temp_privileges[session_id]:
            return False
        
        priv_info = self.temp_privileges[session_id][privilege]
        
        # Check if privilege has expired
        if time.time() > priv_info['expires_at']:
            print(f"⏰ Temporary privilege '{privilege}' has expired")
            del self.temp_privileges[session_id][privilege]
            return False
        
        # Mark as used
        priv_info['used'] = True
        return True
    
    def sensitive_operation(self, session_id, operation_type):
        """Perform operation requiring temporary privilege escalation"""
        required_privilege = f"temp_{operation_type}"
        
        if not self.has_temp_privilege(session_id, required_privilege):
            return f"Access denied: Temporary privilege '{required_privilege}' required"
        
        session = auth_system.active_sessions[session_id]
        username = session['username']
        
        return f"Sensitive operation '{operation_type}' completed by {username} with temporary privileges"

# Demonstrate privilege escalation
priv_system = PrivilegeEscalationSystem()

print("\nPrivilege Escalation System")
print("-" * 30)

# Alice requests privilege escalation for sensitive operation
print("Alice requests privilege escalation:")
success, message = priv_system.request_privilege_escalation(
    alice_session, 
    "temp_delete_system_files", 
    "Cleaning up corrupted backup files"
)
print(f"Request result: {message}\n")

if success:
    # Alice performs the sensitive operation
    print("Alice performs sensitive operation:")
    result = priv_system.sensitive_operation(alice_session, "delete_system_files")
    print(f"Operation result: {result}\n")
    
    # Try to use the same privilege again (should be marked as used)
    print("Alice tries to use same privilege again:")
    result = priv_system.sensitive_operation(alice_session, "delete_system_files")
    print(f"Second attempt: {result}")

# Bob tries to request privilege escalation (should fail)
print("\nBob requests privilege escalation:")
success, message = priv_system.request_privilege_escalation(
    bob_session, 
    "temp_admin_access", 
    "I need admin access"
)
print(f"Request result: {message}")

# Example 5: Authorization Audit Trail

Track all authorization decisions for security monitoring and compliance.

In [ ]:
# Authorization audit system
class AuthorizationAuditor:
    def __init__(self):
        self.audit_log = []
    
    def log_authorization_check(self, session_id, username, permission, granted, details=""):
        """Log every authorization check"""
        log_entry = {
            'timestamp': datetime.now().isoformat(),
            'session_id': session_id,
            'username': username,
            'permission': permission,
            'granted': granted,
            'details': details
        }
        self.audit_log.append(log_entry)
    
    def get_suspicious_activity(self):
        """Identify potentially suspicious authorization patterns"""
        suspicious = []
        
        # Look for users with many denied authorization attempts
        denial_counts = {}
        for entry in self.audit_log:
            if not entry.get('granted', True):  # False or missing (denied)
                username = entry['username']
                denial_counts[username] = denial_counts.get(username, 0) + 1
        
        for username, count in denial_counts.items():
            if count > 5:
                suspicious.append(f"User '{username}' has {count} authorization denials")
        
        return suspicious

# Enhanced authorization system with auditing
class AuditedAuthSystem(SecureAuthorizationSystem):
    def __init__(self):
        super().__init__()
        self.auditor = AuthorizationAuditor()
    
    def has_permission(self, session_id, required_permission):
        """Override to add auditing"""
        if session_id not in self.active_sessions:
            self.auditor.log_authorization_check(
                session_id, "unknown", required_permission, False, "Invalid session"
            )
            return False
        
        session = self.active_sessions[session_id]
        username = session['username']
        user_roles = session['roles']
        
        # Check permissions
        granted = False
        for role in user_roles:
            if role in self.role_permissions:
                if required_permission in self.role_permissions[role]:
                    granted = True
                    break
        
        # Log the authorization check
        self.auditor.log_authorization_check(
            session_id, username, required_permission, granted, 
            f"Roles: {user_roles}"
        )
        
        return granted

# Demonstrate audited authorization
audited_system = AuditedAuthSystem()

print("\nAuthorization Audit Trail")
print("-" * 30)

# Create some test activity
alice_session = audited_system.login('alice', 'secret123')
bob_session = audited_system.login('bob', 'password')

# Generate some authorization events
audited_system.has_permission(alice_session, 'read_posts')
audited_system.has_permission(alice_session, 'create_posts')
audited_system.has_permission(alice_session, 'manage_users')  # Should be denied
audited_system.has_permission(bob_session, 'create_posts')     # Should be denied
audited_system.has_permission(bob_session, 'delete_posts')     # Should be denied
audited_system.has_permission(bob_session, 'manage_users')     # Should be denied

# Show audit trail
print("\nRecent authorization events:")
for entry in audited_system.auditor.audit_log[-6:]:
    status = "✅ GRANTED" if entry['granted'] else "❌ DENIED"
    print(f"  {entry['timestamp'][:19]} | {entry['username']} | {entry['permission']} | {status}")

# Check for suspicious activity
print(f"\nSuspicious activity detection:")
suspicious = audited_system.auditor.get_suspicious_activity()
if suspicious:
    for alert in suspicious:
        print(f"  🚨 {alert}")
else:
    print("  No suspicious activity detected")

# Show user activity summary
bob_activity = [entry for entry in audited_system.auditor.audit_log if entry['username'] == 'bob']
print(f"\nBob's recent activity:")
print(f"  Total authorization checks: {len(bob_activity)}")
denied_count = sum(1 for entry in bob_activity if not entry.get('granted', True))
print(f"  Denied requests: {denied_count}")

# Chapter 4 Summary

## Key Principles:

✅ **Never Trust User Input**:
- Don't accept privilege information from users
- Always check against your own authorization system
- Validate session state on server side

✅ **Proper Authentication First**:
- Authorization requires valid authentication
- Check authentication before checking authorization
- Use secure session management

✅ **Principle of Least Privilege**:
- Grant minimum permissions necessary
- Use temporary privilege escalation when needed
- Regularly review and revoke unused permissions

✅ **Separation of Concerns**:
- Keep authentication and authorization systems distinct
- Use role-based access control (RBAC)
- Implement consistent authorization patterns

✅ **Audit Everything**:
- Log all authorization decisions
- Monitor for suspicious patterns
- Enable compliance and forensics

## Common Vulnerabilities:

❌ **Trusting User-Supplied Authorization**: `?admin=1` parameters
❌ **Missing Authentication Checks**: Checking authorization without authentication
❌ **Privilege Escalation**: Running with unnecessary elevated privileges
❌ **Inconsistent Enforcement**: Some code paths skip authorization
❌ **No Audit Trail**: Can't detect or investigate authorization failures

## Best Practices:

- **Use decorators** for consistent authorization enforcement
- **Implement session-based** authorization systems
- **Apply principle of least privilege** everywhere
- **Log all authorization decisions** for auditing
- **Regularly review** user permissions and roles
- **Test authorization** with different user types

> **Remember**: Authorization is not optional - every privileged operation must check permissions!

# Chapter 4: Authorization
## Controlling What Users Can Do

### Course: TCPRG4005 - Secure Programming

---

## Learning Objectives

1. **Understand** the difference between authentication and authorization
2. **Implement** secure authorization systems
3. **Avoid** trusting user-supplied authorization data
4. **Apply** principle of least privilege
5. **Design** secure privilege escalation patterns

---

## Key Principle

> **Authorization determines what a user can do AFTER they've been authenticated**

Never trust user-provided privilege information - always check against your own system.

In [1]:
# Setup for authorization examples
import json
import time
from datetime import datetime
from enum import Enum
from functools import wraps

print("🔐 Chapter 4: Authorization")
print("Controlling user privileges and access")
print("=" * 45)

🔐 Chapter 4: Authorization
Controlling user privileges and access


# Example 1: Vulnerable Authorization - Trusting User Input

The most common authorization vulnerability: trusting data from the user.

In [2]:
# Demonstrate the classic "?admin=1" vulnerability
class VulnerableSystem:
    def __init__(self):
        self.users = {
            'alice': {'password': 'secret123', 'is_admin': False},
            'bob': {'password': 'password', 'is_admin': False},
            'admin': {'password': 'supersecret', 'is_admin': True}
        }
    
    def vulnerable_check_admin(self, username, user_provided_admin_flag):
        """VULNERABLE: Trusts user-provided admin flag"""
        # BAD: This trusts data from the user
        if user_provided_admin_flag == "1" or user_provided_admin_flag == True:
            print(f"🚨 DANGER: Granting admin access to {username} based on user input!")
            return True
        return False
    
    def secure_check_admin(self, username):
        """SECURE: Checks against internal data only"""
        if username in self.users:
            is_admin = self.users[username]['is_admin']
            print(f"✅ Checking admin status for {username}: {is_admin}")
            return is_admin
        return False

# Demonstrate the vulnerability
system = VulnerableSystem()

print("Authorization Vulnerability Demonstration")
print("-" * 45)

# Normal user tries to access admin function
print("1. Normal access attempt:")
alice_is_admin = system.secure_check_admin('alice')
print(f"   Alice admin status: {alice_is_admin}\n")

# Attacker exploits vulnerable function
print("2. Attack using user-provided admin flag:")
print("   URL: /admin?user=alice&admin=1")
attack_result = system.vulnerable_check_admin('alice', "1")
print(f"   Attack successful: {attack_result}")
print("   🚨 Alice gained admin access without being admin!\n")

# Secure function prevents attack
print("3. Same attack on secure function:")
secure_result = system.secure_check_admin('alice')
print(f"   Attack failed: {secure_result}")
print("   ✅ Secure function only trusts internal data")

Authorization Vulnerability Demonstration
---------------------------------------------
1. Normal access attempt:
✅ Checking admin status for alice: False
   Alice admin status: False

2. Attack using user-provided admin flag:
   URL: /admin?user=alice&admin=1
🚨 DANGER: Granting admin access to alice based on user input!
   Attack successful: True
   🚨 Alice gained admin access without being admin!

3. Same attack on secure function:
✅ Checking admin status for alice: False
   Attack failed: False
   ✅ Secure function only trusts internal data


# Example 2: Session-Based Authorization

Proper authorization should be tied to authenticated sessions, not user input.

In [3]:
# Secure session-based authorization system
class SecureAuthorizationSystem:
    def __init__(self):
        self.users = {
            'alice': {'password': 'secret123', 'roles': ['user', 'editor']},
            'bob': {'password': 'password', 'roles': ['user']},
            'admin': {'password': 'supersecret', 'roles': ['user', 'admin', 'editor']}
        }
        self.active_sessions = {}
        self.role_permissions = {
            'user': ['read_posts', 'comment'],
            'editor': ['read_posts', 'comment', 'create_posts', 'edit_posts'],
            'admin': ['read_posts', 'comment', 'create_posts', 'edit_posts', 'delete_posts', 'manage_users']
        }
    
    def login(self, username, password):
        """Authenticate user and create session"""
        if username in self.users and self.users[username]['password'] == password:
            session_id = f"session_{username}_{int(time.time())}"
            self.active_sessions[session_id] = {
                'username': username,
                'roles': self.users[username]['roles'],
                'login_time': datetime.now()
            }
            return session_id
        return None
    
    def has_permission(self, session_id, required_permission):
        """Check if session has required permission"""
        if session_id not in self.active_sessions:
            print(f"❌ No active session found")
            return False
        
        session = self.active_sessions[session_id]
        username = session['username']
        user_roles = session['roles']
        
        # Check if any of the user's roles grant the required permission
        for role in user_roles:
            if role in self.role_permissions:
                if required_permission in self.role_permissions[role]:
                    print(f"✅ {username} has permission '{required_permission}' via role '{role}'")
                    return True
        
        print(f"❌ {username} lacks permission '{required_permission}'")
        return False
    
    def admin_operation(self, session_id, operation):
        """Perform admin-only operation"""
        if not self.has_permission(session_id, 'manage_users'):
            return "Access denied: Admin privileges required"
        
        return f"Admin operation '{operation}' completed successfully"

# Demonstrate secure authorization
auth_system = SecureAuthorizationSystem()

print("\nSecure Session-Based Authorization")
print("-" * 40)

# Alice logs in (regular user)
alice_session = auth_system.login('alice', 'secret123')
print(f"Alice logged in: {alice_session}\n")

# Alice tries to perform operations
print("Alice's permissions:")
auth_system.has_permission(alice_session, 'read_posts')
auth_system.has_permission(alice_session, 'create_posts') 
auth_system.has_permission(alice_session, 'manage_users')

print("\nAlice tries admin operation:")
result = auth_system.admin_operation(alice_session, "delete_user")
print(f"Result: {result}\n")

# Admin logs in
admin_session = auth_system.login('admin', 'supersecret')
print(f"Admin logged in: {admin_session}")

print("\nAdmin performs same operation:")
result = auth_system.admin_operation(admin_session, "delete_user")
print(f"Result: {result}")


Secure Session-Based Authorization
----------------------------------------
Alice logged in: session_alice_1753564890

Alice's permissions:
✅ alice has permission 'read_posts' via role 'user'
✅ alice has permission 'create_posts' via role 'editor'
❌ alice lacks permission 'manage_users'

Alice tries admin operation:
❌ alice lacks permission 'manage_users'
Result: Access denied: Admin privileges required

Admin logged in: session_admin_1753564890

Admin performs same operation:
✅ admin has permission 'manage_users' via role 'admin'
Result: Admin operation 'delete_user' completed successfully


# Example 3: Authorization Decorators

Use decorators to enforce authorization requirements consistently.

In [4]:
# Authorization decorator pattern
def requires_permission(required_permission):
    """Decorator to enforce permission requirements"""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Extract session_id from arguments
            session_id = kwargs.get('session_id') or (args[0] if args else None)
            
            if not session_id:
                return "Error: No session provided"
            
            # Check permission using our auth system
            if not auth_system.has_permission(session_id, required_permission):
                return f"Access denied: '{required_permission}' permission required"
            
            # Permission granted, execute function
            return func(*args, **kwargs)
        return wrapper
    return decorator

def requires_role(required_role):
    """Decorator to enforce role requirements"""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            session_id = kwargs.get('session_id') or (args[0] if args else None)
            
            if not session_id or session_id not in auth_system.active_sessions:
                return "Error: Invalid session"
            
            session = auth_system.active_sessions[session_id]
            if required_role not in session['roles']:
                return f"Access denied: '{required_role}' role required"
            
            return func(*args, **kwargs)
        return wrapper
    return decorator

# Examples using decorators
@requires_permission('create_posts')
def create_blog_post(session_id, title, content):
    """Create a new blog post"""
    session = auth_system.active_sessions[session_id]
    username = session['username']
    return f"Blog post '{title}' created by {username}"

@requires_permission('delete_posts')
def delete_blog_post(session_id, post_id):
    """Delete a blog post"""
    session = auth_system.active_sessions[session_id]
    username = session['username']
    return f"Blog post {post_id} deleted by {username}"

@requires_role('admin')
def system_backup(session_id):
    """Perform system backup"""
    return "System backup completed successfully"

# Test authorization decorators
print("\nAuthorization Decorators")
print("-" * 25)

# Alice (editor) creates a post
print("Alice creates a blog post:")
result = create_blog_post(alice_session, "My First Post", "Hello world!")
print(f"Result: {result}\n")

# Alice tries to delete a post (needs admin permission)
print("Alice tries to delete a post:")
result = delete_blog_post(alice_session, "post123")
print(f"Result: {result}\n")

# Admin deletes a post
print("Admin deletes a post:")
result = delete_blog_post(admin_session, "post123")
print(f"Result: {result}\n")

# Bob (basic user) tries to create a post
bob_session = auth_system.login('bob', 'password')
print("Bob tries to create a post:")
result = create_blog_post(bob_session, "Bob's Post", "Hello!")
print(f"Result: {result}")


Authorization Decorators
-------------------------
Alice creates a blog post:
✅ alice has permission 'create_posts' via role 'editor'
Result: Blog post 'My First Post' created by alice

Alice tries to delete a post:
❌ alice lacks permission 'delete_posts'
Result: Access denied: 'delete_posts' permission required

Admin deletes a post:
✅ admin has permission 'delete_posts' via role 'admin'
Result: Blog post post123 deleted by admin

Bob tries to create a post:
❌ bob lacks permission 'create_posts'
Result: Access denied: 'create_posts' permission required


# Example 4: Principle of Least Privilege

Grant minimal permissions needed for each operation, use privilege escalation when needed.

In [5]:
# Principle of least privilege implementation
class PrivilegeEscalationSystem:
    def __init__(self):
        self.temp_privileges = {}  # Temporary elevated privileges
        
    def request_privilege_escalation(self, session_id, required_privilege, justification):
        """Request temporary privilege escalation"""
        if session_id not in auth_system.active_sessions:
            return False, "Invalid session"
        
        session = auth_system.active_sessions[session_id]
        username = session['username']
        
        # Simple approval logic (in real systems, this might require another admin)
        if 'admin' in session['roles'] or 'editor' in session['roles']:
            # Grant temporary privilege for 5 minutes
            expiry = time.time() + 300  # 5 minutes
            
            if session_id not in self.temp_privileges:
                self.temp_privileges[session_id] = {}
            
            self.temp_privileges[session_id][required_privilege] = {
                'granted_at': time.time(),
                'expires_at': expiry,
                'justification': justification,
                'used': False
            }
            
            print(f"✅ Temporary privilege '{required_privilege}' granted to {username}")
            print(f"   Justification: {justification}")
            print(f"   Expires in 5 minutes")
            return True, "Privilege escalation approved"
        
        print(f"❌ Privilege escalation denied for {username}")
        return False, "Insufficient base privileges for escalation"
    
    def has_temp_privilege(self, session_id, privilege):
        """Check if session has temporary privilege"""
        if session_id not in self.temp_privileges:
            return False
        
        if privilege not in self.temp_privileges[session_id]:
            return False
        
        priv_info = self.temp_privileges[session_id][privilege]
        
        # Check if privilege has expired
        if time.time() > priv_info['expires_at']:
            print(f"⏰ Temporary privilege '{privilege}' has expired")
            del self.temp_privileges[session_id][privilege]
            return False
        
        # Mark as used
        priv_info['used'] = True
        return True
    
    def sensitive_operation(self, session_id, operation_type):
        """Perform operation requiring temporary privilege escalation"""
        required_privilege = f"temp_{operation_type}"
        
        if not self.has_temp_privilege(session_id, required_privilege):
            return f"Access denied: Temporary privilege '{required_privilege}' required"
        
        session = auth_system.active_sessions[session_id]
        username = session['username']
        
        return f"Sensitive operation '{operation_type}' completed by {username} with temporary privileges"

# Demonstrate privilege escalation
priv_system = PrivilegeEscalationSystem()

print("\nPrivilege Escalation System")
print("-" * 30)

# Alice requests privilege escalation for sensitive operation
print("Alice requests privilege escalation:")
success, message = priv_system.request_privilege_escalation(
    alice_session, 
    "temp_delete_system_files", 
    "Cleaning up corrupted backup files"
)
print(f"Request result: {message}\n")

if success:
    # Alice performs the sensitive operation
    print("Alice performs sensitive operation:")
    result = priv_system.sensitive_operation(alice_session, "delete_system_files")
    print(f"Operation result: {result}\n")
    
    # Try to use the same privilege again (should be marked as used)
    print("Alice tries to use same privilege again:")
    result = priv_system.sensitive_operation(alice_session, "delete_system_files")
    print(f"Second attempt: {result}")

# Bob tries to request privilege escalation (should fail)
print("\nBob requests privilege escalation:")
success, message = priv_system.request_privilege_escalation(
    bob_session, 
    "temp_admin_access", 
    "I need admin access"
)
print(f"Request result: {message}")


Privilege Escalation System
------------------------------
Alice requests privilege escalation:
✅ Temporary privilege 'temp_delete_system_files' granted to alice
   Justification: Cleaning up corrupted backup files
   Expires in 5 minutes
Request result: Privilege escalation approved

Alice performs sensitive operation:
Operation result: Sensitive operation 'delete_system_files' completed by alice with temporary privileges

Alice tries to use same privilege again:
Second attempt: Sensitive operation 'delete_system_files' completed by alice with temporary privileges

Bob requests privilege escalation:
❌ Privilege escalation denied for bob
Request result: Insufficient base privileges for escalation


# Example 5: Authorization Audit Trail

Track all authorization decisions for security monitoring and compliance.

In [6]:
# Authorization audit system
class AuthorizationAuditor:
    def __init__(self):
        self.audit_log = []
    
    def log_authorization_check(self, session_id, username, permission, granted, details=""):
        """Log every authorization check"""
        log_entry = {
            'timestamp': datetime.now().isoformat(),
            'session_id': session_id,
            'username': username,
            'permission': permission,
            'granted': granted,
            'details': details
        }
        self.audit_log.append(log_entry)
    
    def log_privilege_escalation(self, session_id, username, privilege, action, details=""):
        """Log privilege escalation events"""
        log_entry = {
            'timestamp': datetime.now().isoformat(),
            'session_id': session_id,
            'username': username,
            'event_type': 'privilege_escalation',
            'privilege': privilege,
            'action': action,  # 'requested', 'granted', 'used', 'denied'
            'details': details
        }
        self.audit_log.append(log_entry)
    
    def get_user_activity(self, username, hours=24):
        """Get user's authorization activity in last N hours"""
        cutoff_time = datetime.now().timestamp() - (hours * 3600)
        
        user_logs = []
        for entry in self.audit_log:
            if entry['username'] == username:
                entry_time = datetime.fromisoformat(entry['timestamp']).timestamp()
                if entry_time > cutoff_time:
                    user_logs.append(entry)
        
        return user_logs
    
    def get_suspicious_activity(self):
        """Identify potentially suspicious authorization patterns"""
        suspicious = []
        
        # Look for users with many denied authorization attempts
        denial_counts = {}
        for entry in self.audit_log:
            if not entry.get('granted', True):  # False or missing (denied)
                username = entry['username']
                denial_counts[username] = denial_counts.get(username, 0) + 1
        
        for username, count in denial_counts.items():
            if count > 5:
                suspicious.append(f"User '{username}' has {count} authorization denials")
        
        return suspicious

# Enhanced authorization system with auditing
class AuditedAuthSystem(SecureAuthorizationSystem):
    def __init__(self):
        super().__init__()
        self.auditor = AuthorizationAuditor()
    
    def has_permission(self, session_id, required_permission):
        """Override to add auditing"""
        if session_id not in self.active_sessions:
            self.auditor.log_authorization_check(
                session_id, "unknown", required_permission, False, "Invalid session"
            )
            return False
        
        session = self.active_sessions[session_id]
        username = session['username']
        user_roles = session['roles']
        
        # Check permissions
        granted = False
        for role in user_roles:
            if role in self.role_permissions:
                if required_permission in self.role_permissions[role]:
                    granted = True
                    break
        
        # Log the authorization check
        self.auditor.log_authorization_check(
            session_id, username, required_permission, granted, 
            f"Roles: {user_roles}"
        )
        
        return granted

# Demonstrate audited authorization
audited_system = AuditedAuthSystem()

print("\nAuthorization Audit Trail")
print("-" * 30)

# Create some test activity
alice_session = audited_system.login('alice', 'secret123')
bob_session = audited_system.login('bob', 'password')

# Generate some authorization events
audited_system.has_permission(alice_session, 'read_posts')
audited_system.has_permission(alice_session, 'create_posts')
audited_system.has_permission(alice_session, 'manage_users')  # Should be denied
audited_system.has_permission(bob_session, 'create_posts')     # Should be denied
audited_system.has_permission(bob_session, 'delete_posts')     # Should be denied
audited_system.has_permission(bob_session, 'manage_users')     # Should be denied

# Show audit trail
print("\nRecent authorization events:")
for entry in audited_system.auditor.audit_log[-6:]:
    status = "✅ GRANTED" if entry['granted'] else "❌ DENIED"
    print(f"  {entry['timestamp'][:19]} | {entry['username']} | {entry['permission']} | {status}")

# Check for suspicious activity
print(f"\nSuspicious activity detection:")
suspicious = audited_system.auditor.get_suspicious_activity()
if suspicious:
    for alert in suspicious:
        print(f"  🚨 {alert}")
else:
    print("  No suspicious activity detected")

# Show user activity summary
print(f"\nBob's recent activity:")
bob_activity = audited_system.auditor.get_user_activity('bob')
print(f"  Total authorization checks: {len(bob_activity)}")
denied_count = sum(1 for entry in bob_activity if not entry.get('granted', True))
print(f"  Denied requests: {denied_count}")


Authorization Audit Trail
------------------------------

Recent authorization events:
  2025-07-26T16:21:43 | alice | read_posts | ✅ GRANTED
  2025-07-26T16:21:43 | alice | create_posts | ✅ GRANTED
  2025-07-26T16:21:43 | alice | manage_users | ❌ DENIED
  2025-07-26T16:21:43 | bob | create_posts | ❌ DENIED
  2025-07-26T16:21:43 | bob | delete_posts | ❌ DENIED
  2025-07-26T16:21:43 | bob | manage_users | ❌ DENIED

Suspicious activity detection:
  No suspicious activity detected

Bob's recent activity:
  Total authorization checks: 3
  Denied requests: 3


# Chapter 4 Summary

## Key Principles:

✅ **Never Trust User Input**:
- Don't accept privilege information from users
- Always check against your own authorization system
- Validate session state on server side

✅ **Proper Authentication First**:
- Authorization requires valid authentication
- Check authentication before checking authorization
- Use secure session management

✅ **Principle of Least Privilege**:
- Grant minimum permissions necessary
- Use temporary privilege escalation when needed
- Regularly review and revoke unused permissions

✅ **Separation of Concerns**:
- Keep authentication and authorization systems distinct
- Use role-based access control (RBAC)
- Implement consistent authorization patterns

✅ **Audit Everything**:
- Log all authorization decisions
- Monitor for suspicious patterns
- Enable compliance and forensics

## Common Vulnerabilities:

❌ **Trusting User-Supplied Authorization**: `?admin=1` parameters
❌ **Missing Authentication Checks**: Checking authorization without authentication
❌ **Privilege Escalation**: Running with unnecessary elevated privileges
❌ **Inconsistent Enforcement**: Some code paths skip authorization
❌ **No Audit Trail**: Can't detect or investigate authorization failures

## Best Practices:

- **Use decorators** for consistent authorization enforcement
- **Implement session-based** authorization systems
- **Apply principle of least privilege** everywhere
- **Log all authorization decisions** for auditing
- **Regularly review** user permissions and roles
- **Test authorization** with different user types

> **Remember**: Authorization is not optional - every privileged operation must check permissions!

# Chapter 4: Authorization
## Controlling What Users Can Do

### Course: TCPRG4005 - Secure Programming

---

## Learning Objectives

1. **Understand** the difference between authentication and authorization
2. **Implement** secure authorization systems
3. **Avoid** trusting user-supplied authorization data
4. **Apply** principle of least privilege
5. **Design** secure privilege escalation patterns

---

## Key Principle

> **Authorization determines what a user can do AFTER they've been authenticated**

Never trust user-provided privilege information - always check against your own system.

In [ ]:
# Setup for authorization examples
import json
import time
from datetime import datetime
from enum import Enum
from functools import wraps

print("🔐 Chapter 4: Authorization")
print("Controlling user privileges and access")
print("=" * 45)

# Example 1: Vulnerable Authorization - Trusting User Input

The most common authorization vulnerability: trusting data from the user.

In [ ]:
# Demonstrate the classic "?admin=1" vulnerability
class VulnerableSystem:
    def __init__(self):
        self.users = {
            'alice': {'password': 'secret123', 'is_admin': False},
            'bob': {'password': 'password', 'is_admin': False},
            'admin': {'password': 'supersecret', 'is_admin': True}
        }
    
    def vulnerable_check_admin(self, username, user_provided_admin_flag):
        """VULNERABLE: Trusts user-provided admin flag"""
        # BAD: This trusts data from the user
        if user_provided_admin_flag == "1" or user_provided_admin_flag == True:
            print(f"🚨 DANGER: Granting admin access to {username} based on user input!")
            return True
        return False
    
    def secure_check_admin(self, username):
        """SECURE: Checks against internal data only"""
        if username in self.users:
            is_admin = self.users[username]['is_admin']
            print(f"✅ Checking admin status for {username}: {is_admin}")
            return is_admin
        return False

# Demonstrate the vulnerability
system = VulnerableSystem()

print("Authorization Vulnerability Demonstration")
print("-" * 45)

# Normal user tries to access admin function
print("1. Normal access attempt:")
alice_is_admin = system.secure_check_admin('alice')
print(f"   Alice admin status: {alice_is_admin}\n")

# Attacker exploits vulnerable function
print("2. Attack using user-provided admin flag:")
print("   URL: /admin?user=alice&admin=1")
attack_result = system.vulnerable_check_admin('alice', "1")
print(f"   Attack successful: {attack_result}")
print("   🚨 Alice gained admin access without being admin!\n")

# Secure function prevents attack
print("3. Same attack on secure function:")
secure_result = system.secure_check_admin('alice')
print(f"   Attack failed: {secure_result}")
print("   ✅ Secure function only trusts internal data")

# Example 2: Session-Based Authorization

Proper authorization should be tied to authenticated sessions, not user input.

In [ ]:
# Secure session-based authorization system
class SecureAuthorizationSystem:
    def __init__(self):
        self.users = {
            'alice': {'password': 'secret123', 'roles': ['user', 'editor']},
            'bob': {'password': 'password', 'roles': ['user']},
            'admin': {'password': 'supersecret', 'roles': ['user', 'admin', 'editor']}
        }
        self.active_sessions = {}
        self.role_permissions = {
            'user': ['read_posts', 'comment'],
            'editor': ['read_posts', 'comment', 'create_posts', 'edit_posts'],
            'admin': ['read_posts', 'comment', 'create_posts', 'edit_posts', 'delete_posts', 'manage_users']
        }
    
    def login(self, username, password):
        """Authenticate user and create session"""
        if username in self.users and self.users[username]['password'] == password:
            session_id = f"session_{username}_{int(time.time())}"
            self.active_sessions[session_id] = {
                'username': username,
                'roles': self.users[username]['roles'],
                'login_time': datetime.now()
            }
            return session_id
        return None
    
    def has_permission(self, session_id, required_permission):
        """Check if session has required permission"""
        if session_id not in self.active_sessions:
            print(f"❌ No active session found")
            return False
        
        session = self.active_sessions[session_id]
        username = session['username']
        user_roles = session['roles']
        
        # Check if any of the user's roles grant the required permission
        for role in user_roles:
            if role in self.role_permissions:
                if required_permission in self.role_permissions[role]:
                    print(f"✅ {username} has permission '{required_permission}' via role '{role}'")
                    return True
        
        print(f"❌ {username} lacks permission '{required_permission}'")
        return False
    
    def admin_operation(self, session_id, operation):
        """Perform admin-only operation"""
        if not self.has_permission(session_id, 'manage_users'):
            return "Access denied: Admin privileges required"
        
        return f"Admin operation '{operation}' completed successfully"

# Demonstrate secure authorization
auth_system = SecureAuthorizationSystem()

print("\nSecure Session-Based Authorization")
print("-" * 40)

# Alice logs in (regular user)
alice_session = auth_system.login('alice', 'secret123')
print(f"Alice logged in: {alice_session}\n")

# Alice tries to perform operations
print("Alice's permissions:")
auth_system.has_permission(alice_session, 'read_posts')
auth_system.has_permission(alice_session, 'create_posts') 
auth_system.has_permission(alice_session, 'manage_users')

print("\nAlice tries admin operation:")
result = auth_system.admin_operation(alice_session, "delete_user")
print(f"Result: {result}\n")

# Admin logs in
admin_session = auth_system.login('admin', 'supersecret')
print(f"Admin logged in: {admin_session}")

print("\nAdmin performs same operation:")
result = auth_system.admin_operation(admin_session, "delete_user")
print(f"Result: {result}")

# Example 3: Authorization Decorators

Use decorators to enforce authorization requirements consistently.

In [ ]:
# Authorization decorator pattern
def requires_permission(required_permission):
    """Decorator to enforce permission requirements"""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Extract session_id from arguments
            session_id = kwargs.get('session_id') or (args[0] if args else None)
            
            if not session_id:
                return "Error: No session provided"
            
            # Check permission using our auth system
            if not auth_system.has_permission(session_id, required_permission):
                return f"Access denied: '{required_permission}' permission required"
            
            # Permission granted, execute function
            return func(*args, **kwargs)
        return wrapper
    return decorator

def requires_role(required_role):
    """Decorator to enforce role requirements"""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            session_id = kwargs.get('session_id') or (args[0] if args else None)
            
            if not session_id or session_id not in auth_system.active_sessions:
                return "Error: Invalid session"
            
            session = auth_system.active_sessions[session_id]
            if required_role not in session['roles']:
                return f"Access denied: '{required_role}' role required"
            
            return func(*args, **kwargs)
        return wrapper
    return decorator

# Examples using decorators
@requires_permission('create_posts')
def create_blog_post(session_id, title, content):
    """Create a new blog post"""
    session = auth_system.active_sessions[session_id]
    username = session['username']
    return f"Blog post '{title}' created by {username}"

@requires_permission('delete_posts')
def delete_blog_post(session_id, post_id):
    """Delete a blog post"""
    session = auth_system.active_sessions[session_id]
    username = session['username']
    return f"Blog post {post_id} deleted by {username}"

@requires_role('admin')
def system_backup(session_id):
    """Perform system backup"""
    return "System backup completed successfully"

# Test authorization decorators
print("\nAuthorization Decorators")
print("-" * 25)

# Alice (editor) creates a post
print("Alice creates a blog post:")
result = create_blog_post(alice_session, "My First Post", "Hello world!")
print(f"Result: {result}\n")

# Alice tries to delete a post (needs admin permission)
print("Alice tries to delete a post:")
result = delete_blog_post(alice_session, "post123")
print(f"Result: {result}\n")

# Admin deletes a post
print("Admin deletes a post:")
result = delete_blog_post(admin_session, "post123")
print(f"Result: {result}\n")

# Bob (basic user) tries to create a post
bob_session = auth_system.login('bob', 'password')
print("Bob tries to create a post:")
result = create_blog_post(bob_session, "Bob's Post", "Hello!")
print(f"Result: {result}")

# Example 4: Principle of Least Privilege

Grant minimal permissions needed for each operation, use privilege escalation when needed.

In [ ]:
# Principle of least privilege implementation
class PrivilegeEscalationSystem:
    def __init__(self):
        self.temp_privileges = {}  # Temporary elevated privileges
        
    def request_privilege_escalation(self, session_id, required_privilege, justification):
        """Request temporary privilege escalation"""
        if session_id not in auth_system.active_sessions:
            return False, "Invalid session"
        
        session = auth_system.active_sessions[session_id]
        username = session['username']
        
        # Simple approval logic (in real systems, this might require another admin)
        if 'admin' in session['roles'] or 'editor' in session['roles']:
            # Grant temporary privilege for 5 minutes
            expiry = time.time() + 300  # 5 minutes
            
            if session_id not in self.temp_privileges:
                self.temp_privileges[session_id] = {}
            
            self.temp_privileges[session_id][required_privilege] = {
                'granted_at': time.time(),
                'expires_at': expiry,
                'justification': justification,
                'used': False
            }
            
            print(f"✅ Temporary privilege '{required_privilege}' granted to {username}")
            print(f"   Justification: {justification}")
            print(f"   Expires in 5 minutes")
            return True, "Privilege escalation approved"
        
        print(f"❌ Privilege escalation denied for {username}")
        return False, "Insufficient base privileges for escalation"
    
    def has_temp_privilege(self, session_id, privilege):
        """Check if session has temporary privilege"""
        if session_id not in self.temp_privileges:
            return False
        
        if privilege not in self.temp_privileges[session_id]:
            return False
        
        priv_info = self.temp_privileges[session_id][privilege]
        
        # Check if privilege has expired
        if time.time() > priv_info['expires_at']:
            print(f"⏰ Temporary privilege '{privilege}' has expired")
            del self.temp_privileges[session_id][privilege]
            return False
        
        # Mark as used
        priv_info['used'] = True
        return True
    
    def sensitive_operation(self, session_id, operation_type):
        """Perform operation requiring temporary privilege escalation"""
        required_privilege = f"temp_{operation_type}"
        
        if not self.has_temp_privilege(session_id, required_privilege):
            return f"Access denied: Temporary privilege '{required_privilege}' required"
        
        session = auth_system.active_sessions[session_id]
        username = session['username']
        
        return f"Sensitive operation '{operation_type}' completed by {username} with temporary privileges"

# Demonstrate privilege escalation
priv_system = PrivilegeEscalationSystem()

print("\nPrivilege Escalation System")
print("-" * 30)

# Alice requests privilege escalation for sensitive operation
print("Alice requests privilege escalation:")
success, message = priv_system.request_privilege_escalation(
    alice_session, 
    "temp_delete_system_files", 
    "Cleaning up corrupted backup files"
)
print(f"Request result: {message}\n")

if success:
    # Alice performs the sensitive operation
    print("Alice performs sensitive operation:")
    result = priv_system.sensitive_operation(alice_session, "delete_system_files")
    print(f"Operation result: {result}\n")
    
    # Try to use the same privilege again (should be marked as used)
    print("Alice tries to use same privilege again:")
    result = priv_system.sensitive_operation(alice_session, "delete_system_files")
    print(f"Second attempt: {result}")

# Bob tries to request privilege escalation (should fail)
print("\nBob requests privilege escalation:")
success, message = priv_system.request_privilege_escalation(
    bob_session, 
    "temp_admin_access", 
    "I need admin access"
)
print(f"Request result: {message}")

# Example 5: Authorization Audit Trail

Track all authorization decisions for security monitoring and compliance.

In [ ]:
# Authorization audit system
class AuthorizationAuditor:
    def __init__(self):
        self.audit_log = []
    
    def log_authorization_check(self, session_id, username, permission, granted, details=""):
        """Log every authorization check"""
        log_entry = {
            'timestamp': datetime.now().isoformat(),
            'session_id': session_id,
            'username': username,
            'permission': permission,
            'granted': granted,
            'details': details
        }
        self.audit_log.append(log_entry)
    
    def log_privilege_escalation(self, session_id, username, privilege, action, details=""):
        """Log privilege escalation events"""
        log_entry = {
            'timestamp': datetime.now().isoformat(),
            'session_id': session_id,
            'username': username,
            'event_type': 'privilege_escalation',
            'privilege': privilege,
            'action': action,  # 'requested', 'granted', 'used', 'denied'
            'details': details
        }
        self.audit_log.append(log_entry)
    
    def get_user_activity(self, username, hours=24):
        """Get user's authorization activity in last N hours"""
        cutoff_time = datetime.now().timestamp() - (hours * 3600)
        
        user_logs = []
        for entry in self.audit_log:
            if entry['username'] == username:
                entry_time = datetime.fromisoformat(entry['timestamp']).timestamp()
                if entry_time > cutoff_time:
                    user_logs.append(entry)
        
        return user_logs
    
    def get_suspicious_activity(self):
        """Identify potentially suspicious authorization patterns"""
        suspicious = []
        
        # Look for users with many denied authorization attempts
        denial_counts = {}
        for entry in self.audit_log:
            if not entry.get('granted', True):  # False or missing (denied)
                username = entry['username']
                denial_counts[username] = denial_counts.get(username, 0) + 1
        
        for username, count in denial_counts.items():
            if count > 5:
                suspicious.append(f"User '{username}' has {count} authorization denials")
        
        return suspicious

# Enhanced authorization system with auditing
class AuditedAuthSystem(SecureAuthorizationSystem):
    def __init__(self):
        super().__init__()
        self.auditor = AuthorizationAuditor()
    
    def has_permission(self, session_id, required_permission):
        """Override to add auditing"""
        if session_id not in self.active_sessions:
            self.auditor.log_authorization_check(
                session_id, "unknown", required_permission, False, "Invalid session"
            )
            return False
        
        session = self.active_sessions[session_id]
        username = session['username']
        user_roles = session['roles']
        
        # Check permissions
        granted = False
        for role in user_roles:
            if role in self.role_permissions:
                if required_permission in self.role_permissions[role]:
                    granted = True
                    break
        
        # Log the authorization check
        self.auditor.log_authorization_check(
            session_id, username, required_permission, granted, 
            f"Roles: {user_roles}"
        )
        
        return granted

# Demonstrate audited authorization
audited_system = AuditedAuthSystem()

print("\nAuthorization Audit Trail")
print("-" * 30)

# Create some test activity
alice_session = audited_system.login('alice', 'secret123')
bob_session = audited_system.login('bob', 'password')

# Generate some authorization events
audited_system.has_permission(alice_session, 'read_posts')
audited_system.has_permission(alice_session, 'create_posts')
audited_system.has_permission(alice_session, 'manage_users')  # Should be denied
audited_system.has_permission(bob_session, 'create_posts')     # Should be denied
audited_system.has_permission(bob_session, 'delete_posts')     # Should be denied
audited_system.has_permission(bob_session, 'manage_users')     # Should be denied

# Show audit trail
print("\nRecent authorization events:")
for entry in audited_system.auditor.audit_log[-6:]:
    status = "✅ GRANTED" if entry['granted'] else "❌ DENIED"
    print(f"  {entry['timestamp'][:19]} | {entry['username']} | {entry['permission']} | {status}")

# Check for suspicious activity
print(f"\nSuspicious activity detection:")
suspicious = audited_system.auditor.get_suspicious_activity()
if suspicious:
    for alert in suspicious:
        print(f"  🚨 {alert}")
else:
    print("  No suspicious activity detected")

# Show user activity summary
print(f"\nBob's recent activity:")
bob_activity = audited_system.auditor.get_user_activity('bob')
print(f"  Total authorization checks: {len(bob_activity)}")
denied_count = sum(1 for entry in bob_activity if not entry.get('granted', True))
print(f"  Denied requests: {denied_count}")

# Chapter 4 Summary

## Key Principles:

✅ **Never Trust User Input**:
- Don't accept privilege information from users
- Always check against your own authorization system
- Validate session state on server side

✅ **Proper Authentication First**:
- Authorization requires valid authentication
- Check authentication before checking authorization
- Use secure session management

✅ **Principle of Least Privilege**:
- Grant minimum permissions necessary
- Use temporary privilege escalation when needed
- Regularly review and revoke unused permissions

✅ **Separation of Concerns**:
- Keep authentication and authorization systems distinct
- Use role-based access control (RBAC)
- Implement consistent authorization patterns

✅ **Audit Everything**:
- Log all authorization decisions
- Monitor for suspicious patterns
- Enable compliance and forensics

## Common Vulnerabilities:

❌ **Trusting User-Supplied Authorization**: `?admin=1` parameters
❌ **Missing Authentication Checks**: Checking authorization without authentication
❌ **Privilege Escalation**: Running with unnecessary elevated privileges
❌ **Inconsistent Enforcement**: Some code paths skip authorization
❌ **No Audit Trail**: Can't detect or investigate authorization failures

## Best Practices:

- **Use decorators** for consistent authorization enforcement
- **Implement session-based** authorization systems
- **Apply principle of least privilege** everywhere
- **Log all authorization decisions** for auditing
- **Regularly review** user permissions and roles
- **Test authorization** with different user types

> **Remember**: Authorization is not optional - every privileged operation must check permissions!

# Chapter 4: Authorization Fundamentals
## TCPRG4005 Secure Programming - Interactive Learning Notebook

**Course**: TCPRG4005-2023-03-01 Secure Programming  
**Chapter**: 4 - Authorization  
**Date**: July 23, 2025

---

### Learning Objectives

By the end of this section, you will be able to:
1. Distinguish between authentication and authorization
2. Identify common authorization vulnerabilities
3. Implement secure authorization checks
4. Design role-based access control (RBAC) systems
5. Prevent privilege escalation attacks
6. Apply defense-in-depth authorization strategies

### Prerequisites
- Understanding of authentication fundamentals (Chapter 3)
- Basic web application concepts
- Knowledge of server-side programming
- Understanding of database interactions

## Section 1: Authorization Security Demonstrations

Authorization controls what authenticated users are allowed to do in our systems. This section provides interactive demonstrations of authorization concepts, vulnerabilities, and secure implementations to accompany the presentation materials.

In [ ]:
import os
import sys
import time
import json
import sqlite3
import hashlib
import secrets
import logging
from datetime import datetime, timedelta
from dataclasses import dataclass
from typing import Dict, Set, List, Optional, Any
import matplotlib.pyplot as plt
import numpy as np

# Set up logging for authorization events
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger('authorization_demo')

print("Chapter 4: Authorization Fundamentals - Section 1")
print("=" * 50)
print("Setting up authorization security demonstrations...")
print("✅ Environment ready for interactive authorization learning!")

### 1.1 Understanding Authentication vs Authorization

Let's start with a clear demonstration of the difference between authentication and authorization through a practical example.

In [ ]:
class User:
    """Represents a user in our system"""
    def __init__(self, user_id: int, username: str, role: str, department: str = None):
        self.user_id = user_id
        self.username = username
        self.role = role
        self.department = department
        self.authenticated = False
        self.login_time = None
    
    def authenticate(self, password: str) -> bool:
        """Simulate authentication process"""
        # In reality, this would verify password hash
        if password == f"{self.username}_password":
            self.authenticated = True
            self.login_time = datetime.now()
            return True
        return False
    
    def __str__(self):
        auth_status = "✅ Authenticated" if self.authenticated else "❌ Not Authenticated"
        return f"User: {self.username} | Role: {self.role} | {auth_status}"

# Create sample users with different roles
users = {
    1: User(1, "alice", "admin", "IT"),
    2: User(2, "bob", "manager", "Sales"),
    3: User(3, "charlie", "employee", "Sales"),
    4: User(4, "diana", "employee", "HR"),
    5: User(5, "eve", "viewer", "Marketing")
}

print("🏢 Company User Directory")
print("=" * 30)
for user in users.values():
    print(f"{user}")

print(f"\n🔐 Let's authenticate some users...")

# Demonstrate authentication
alice = users[1]
bob = users[2]
charlie = users[3]

print(f"\n👤 Alice attempting login...")
if alice.authenticate("alice_password"):
    print(f"✅ {alice.username} authenticated successfully!")
else:
    print(f"❌ Authentication failed for {alice.username}")

print(f"\n👤 Bob attempting login with wrong password...")
if bob.authenticate("wrong_password"):
    print(f"✅ {bob.username} authenticated successfully!")
else:
    print(f"❌ Authentication failed for {bob.username}")

print(f"\n👤 Bob attempting login with correct password...")
if bob.authenticate("bob_password"):
    print(f"✅ {bob.username} authenticated successfully!")
else:
    print(f"❌ Authentication failed for {bob.username}")

print(f"\n📊 Current Authentication Status:")
for user in users.values():
    print(f"  {user}")

### 1.2 Basic Authorization System Implementation

Now let's implement a simple but secure authorization system that demonstrates proper separation of concerns.

In [ ]:
class AuthorizationSystem:
    """Secure authorization system implementing RBAC"""
    
    def __init__(self):
        # Define role-based permissions
        self.role_permissions = {
            'admin': {
                'read_user', 'create_user', 'update_user', 'delete_user',
                'read_system', 'update_system', 'read_reports', 'manage_roles'
            },
            'manager': {
                'read_user', 'create_user', 'update_user',
                'read_reports', 'manage_team'
            },
            'employee': {
                'read_user', 'update_own_profile', 'read_reports'
            },
            'viewer': {
                'read_user', 'read_reports'
            }
        }
        
        # Track authorization events
        self.auth_log = []
        
    def check_permission(self, user: User, permission: str, target_user_id: int = None) -> bool:
        """
        Check if user has permission to perform action
        """
        # Step 1: Verify user is authenticated
        if not user.authenticated:
            self._log_authorization("DENIED", user.username, permission, "Not authenticated")
            return False
        
        # Step 2: Get user's permissions based on role
        user_permissions = self.role_permissions.get(user.role, set())
        
        # Step 3: Check if user has the required permission
        if permission not in user_permissions:
            self._log_authorization("DENIED", user.username, permission, "Insufficient role permissions")
            return False
        
        # Step 4: Additional business logic checks
        if permission == 'update_own_profile' and target_user_id:
            if user.user_id != target_user_id:
                self._log_authorization("DENIED", user.username, permission, "Can only update own profile")
                return False
        
        # Permission granted
        self._log_authorization("GRANTED", user.username, permission, "Success")
        return True
    
    def _log_authorization(self, decision: str, username: str, permission: str, reason: str):
        """Log authorization decisions for audit trail"""
        log_entry = {
            'timestamp': datetime.now().isoformat(),
            'decision': decision,
            'user': username,
            'permission': permission,
            'reason': reason
        }
        self.auth_log.append(log_entry)
        
        # Also log to system logger
        if decision == "GRANTED":
            logger.info(f"Authorization {decision}: {username} -> {permission} ({reason})")
        else:
            logger.warning(f"Authorization {decision}: {username} -> {permission} ({reason})")
    
    def get_user_permissions(self, user: User) -> Set[str]:
        """Get all permissions for a user"""
        if not user.authenticated:
            return set()
        return self.role_permissions.get(user.role, set())
    
    def show_authorization_log(self, last_n: int = 10):
        """Display recent authorization events"""
        print(f"\n📋 Authorization Audit Log (Last {last_n} events)")
        print("-" * 70)
        
        recent_logs = self.auth_log[-last_n:]
        for entry in recent_logs:
            status_icon = "✅" if entry['decision'] == "GRANTED" else "❌"
            print(f"{status_icon} {entry['timestamp'][:19]} | {entry['user']:<10} | "
                  f"{entry['permission']:<20} | {entry['reason']}")

# Initialize authorization system
auth_system = AuthorizationSystem()

print("🔐 Authorization System Initialized")
print("=" * 40)
print("Available permissions by role:")
for role, permissions in auth_system.role_permissions.items():
    print(f"\n{role.upper()}:")
    for perm in sorted(permissions):
        print(f"  • {perm}")

### 1.3 Demonstration: Secure Authorization in Action

Let's test our authorization system with various scenarios to see how proper authorization works.

In [ ]:
def demonstrate_authorization_scenarios():
    """Test various authorization scenarios"""
    
    print("🎭 Authorization Scenarios Demonstration")
    print("=" * 50)
    
    # Get our authenticated users
    alice = users[1]  # admin
    bob = users[2]    # manager  
    charlie = users[3]  # employee (not authenticated yet)
    
    scenarios = [
        # Scenario 1: Admin accessing system functions
        (alice, "read_system", None, "Admin reading system configuration"),
        (alice, "delete_user", None, "Admin deleting a user account"),
        (alice, "manage_roles", None, "Admin managing user roles"),
        
        # Scenario 2: Manager accessing appropriate functions
        (bob, "read_reports", None, "Manager reading department reports"),
        (bob, "create_user", None, "Manager creating new team member"),
        (bob, "delete_user", None, "Manager trying to delete users (should fail)"),
        
        # Scenario 3: Unauthenticated user attempts
        (charlie, "read_user", None, "Unauthenticated user trying to read data"),
        
        # Scenario 4: Self-service operations
        (bob, "update_own_profile", bob.user_id, "User updating own profile"),
        (bob, "update_own_profile", alice.user_id, "User trying to update someone else's profile"),
    ]
    
    for i, (user, permission, target_id, description) in enumerate(scenarios, 1):
        print(f"\n🔍 Scenario {i}: {description}")
        print(f"   User: {user.username} ({user.role})")
        print(f"   Permission: {permission}")
        
        # Check authorization
        is_authorized = auth_system.check_permission(user, permission, target_id)
        
        if is_authorized:
            print(f"   Result: ✅ AUTHORIZED - Action permitted")
        else:
            print(f"   Result: ❌ DENIED - Access forbidden")
    
    # Show the audit log
    auth_system.show_authorization_log()

# Run the demonstration
demonstrate_authorization_scenarios()

### 1.4 Vulnerability Demonstration: Trusting User-Supplied Data

Now let's demonstrate the dangerous vulnerability of trusting user-supplied authorization data.

In [ ]:
class VulnerableAuthSystem:
    """
    ⚠️ WARNING: This is intentionally vulnerable for educational purposes!
    DO NOT use this approach in production systems!
    """
    
    def __init__(self):
        self.users = {
            "alice": {"password": "alice_pass", "role": "user"},
            "admin": {"password": "admin_pass", "role": "admin"}
        }
    
    def vulnerable_cookie_auth(self, username: str, password: str, user_role: str = None):
        """BAD: Trusting role from user input (like cookies)"""
        print(f"\n🔓 Vulnerable Cookie-Based Authorization")
        print(f"Username: {username}")
        print(f"Password: {'*' * len(password)}")
        print(f"Role (from cookie): {user_role}")
        
        # Authenticate user
        if username in self.users and self.users[username]["password"] == password:
            print("✅ Authentication successful")
            
            # VULNERABILITY: Trusting user-supplied role data!
            if user_role == "admin":
                print("🚨 ADMIN ACCESS GRANTED - Deleting all users...")
                return True, "admin"
            else:
                print("👤 Regular user access")
                return True, "user"
        else:
            print("❌ Authentication failed")
            return False, None
    
    def vulnerable_url_auth(self, username: str, admin_flag: str = "0"):
        """BAD: Trusting admin flag from URL parameters"""
        print(f"\n🔓 Vulnerable URL Parameter Authorization")
        print(f"URL: /dashboard?user={username}&admin={admin_flag}")
        
        # VULNERABILITY: Trusting URL parameters for authorization!
        if admin_flag == "1":
            print("🚨 ADMIN ACCESS GRANTED via URL parameter!")
            print("   Showing secret admin panel...")
            return "admin"
        else:
            print("👤 Regular user dashboard")
            return "user"
    
    def vulnerable_hidden_field_auth(self, form_data: dict):
        """BAD: Trusting hidden form fields"""
        print(f"\n🔓 Vulnerable Hidden Field Authorization")
        print(f"Form data received: {form_data}")
        
        # VULNERABILITY: Trusting hidden form fields!
        if form_data.get("is_admin") == "true":
            print("🚨 ADMIN ACCESS GRANTED via hidden field!")
            print("   Processing admin action...")
            return "admin"
        else:
            print("👤 Regular user action")
            return "user"

# Demonstrate vulnerable authorization
vuln_system = VulnerableAuthSystem()

print("⚠️  VULNERABILITY DEMONSTRATIONS")
print("=" * 40)
print("These examples show common authorization mistakes!")

# Demo 1: Cookie-based vulnerability
print("\n" + "="*60)
print("DEMO 1: Cookie Role Injection Attack")
print("="*60)

# Normal user login
success, role = vuln_system.vulnerable_cookie_auth("alice", "alice_pass", "user")
print(f"Result: {role} access granted")

# Attacker modifies cookie to claim admin role
print(f"\n🔴 ATTACK: User modifies cookie to set role=admin")
success, role = vuln_system.vulnerable_cookie_auth("alice", "alice_pass", "admin")
print(f"Result: {role} access granted - UNAUTHORIZED ADMIN ACCESS!")

# Demo 2: URL parameter vulnerability  
print("\n" + "="*60)
print("DEMO 2: URL Parameter Manipulation Attack")
print("="*60)

# Normal access
access_level = vuln_system.vulnerable_url_auth("alice", "0")
print(f"Result: {access_level} access")

# Attacker modifies URL
print(f"\n🔴 ATTACK: User changes URL to ?admin=1")
access_level = vuln_system.vulnerable_url_auth("alice", "1")
print(f"Result: {access_level} access - UNAUTHORIZED ADMIN ACCESS!")

# Demo 3: Hidden field vulnerability
print("\n" + "="*60)
print("DEMO 3: Hidden Form Field Manipulation Attack")
print("="*60)

# Normal form submission
normal_form = {"action": "update_profile", "is_admin": "false"}
access_level = vuln_system.vulnerable_hidden_field_auth(normal_form)
print(f"Result: {access_level} access")

# Attacker modifies hidden field
print(f"\n🔴 ATTACK: User modifies hidden field with browser dev tools")
malicious_form = {"action": "delete_all_users", "is_admin": "true"}
access_level = vuln_system.vulnerable_hidden_field_auth(malicious_form)
print(f"Result: {access_level} access - UNAUTHORIZED ADMIN ACCESS!")

print(f"\n💡 Key Lesson: NEVER trust authorization data that comes from the client!")

### 1.5 Secure Authorization Implementation Patterns

Now let's see the correct way to implement authorization that prevents these vulnerabilities.

In [ ]:
class SecureAuthSystem:
    """Demonstrates secure authorization patterns"""
    
    def __init__(self):
        # Server-side user database (trusted source of truth)
        self.users_db = {
            1: {"username": "alice", "password_hash": self._hash_password("alice_pass"), "role": "user"},
            2: {"username": "admin", "password_hash": self._hash_password("admin_pass"), "role": "admin"}
        }
        
        # Active sessions (server-side only)
        self.active_sessions = {}
        
    def _hash_password(self, password: str) -> str:
        """Simple password hashing for demo"""
        return hashlib.sha256(password.encode()).hexdigest()
    
    def login(self, username: str, password: str) -> Optional[str]:
        """Secure login that establishes server-side session"""
        password_hash = self._hash_password(password)
        
        # Find user by username and verify password
        for user_id, user_data in self.users_db.items():
            if (user_data["username"] == username and 
                user_data["password_hash"] == password_hash):
                
                # Create secure session token
                session_token = secrets.token_urlsafe(32)
                self.active_sessions[session_token] = {
                    "user_id": user_id,
                    "username": username,
                    "role": user_data["role"],
                    "login_time": datetime.now()
                }
                
                print(f"✅ Login successful for {username}")
                print(f"🎫 Session token: {session_token[:20]}...")
                return session_token
        
        print(f"❌ Login failed for {username}")
        return None
    
    def check_authorization(self, session_token: str, required_permission: str) -> bool:
        """Secure authorization check using server-side session data"""
        
        # Step 1: Validate session token
        if session_token not in self.active_sessions:
            print(f"❌ Invalid or expired session token")
            return False
        
        session_data = self.active_sessions[session_token]
        username = session_data["username"]
        role = session_data["role"]
        
        print(f"🔍 Authorization check for {username} ({role})")
        print(f"   Required permission: {required_permission}")
        
        # Step 2: Check role-based permissions (server-side logic only)
        role_permissions = {
            "admin": ["read_users", "create_user", "delete_user", "admin_panel"],
            "user": ["read_own_profile", "update_own_profile"]
        }
        
        user_permissions = role_permissions.get(role, [])
        
        if required_permission in user_permissions:
            print(f"   ✅ Permission granted")
            return True
        else:
            print(f"   ❌ Permission denied - insufficient privileges")
            return False
    
    def secure_admin_action(self, session_token: str, action: str):
        """Example of properly protected admin function"""
        print(f"\n🔒 Attempting admin action: {action}")
        
        if self.check_authorization(session_token, "admin_panel"):
            print(f"   🎯 Executing admin action: {action}")
            return True
        else:
            print(f"   🚫 Admin action blocked: insufficient privileges")
            return False
    
    def secure_user_action(self, session_token: str, action: str):
        """Example of properly protected user function"""
        print(f"\n🔒 Attempting user action: {action}")
        
        if self.check_authorization(session_token, "read_own_profile"):
            print(f"   📝 Executing user action: {action}")
            return True
        else:
            print(f"   🚫 User action blocked: insufficient privileges")
            return False

# Demonstrate secure authorization
secure_system = SecureAuthSystem()

print("🛡️  SECURE AUTHORIZATION DEMONSTRATION")
print("=" * 50)

# Step 1: Users log in and get session tokens
print("Step 1: User Authentication")
print("-" * 30)

alice_token = secure_system.login("alice", "alice_pass")
admin_token = secure_system.login("admin", "admin_pass")

print(f"\nStep 2: Authorization Testing")
print("-" * 30)

# Test normal user actions
print(f"\n👤 Alice (regular user) attempting actions:")
secure_system.secure_user_action(alice_token, "view_profile")
secure_system.secure_admin_action(alice_token, "delete_all_users")

# Test admin actions
print(f"\n👑 Admin attempting actions:")
secure_system.secure_user_action(admin_token, "view_profile")
secure_system.secure_admin_action(admin_token, "delete_all_users")

# Test with invalid token (simulating attacker)
print(f"\n🔴 Attacker with fake token:")
fake_token = "fake_token_12345"
secure_system.secure_admin_action(fake_token, "delete_all_users")

print(f"\n🔐 Key Security Features:")
print("  ✅ Session tokens generated server-side")
print("  ✅ Role data stored server-side only")
print("  ✅ Authorization decisions made server-side")
print("  ✅ No trust in client-supplied data")

### 1.6 Insecure Direct Object Reference (IDOR) Demonstration

One of the most common authorization vulnerabilities is failing to check if a user can access specific objects.

In [ ]:
class DocumentSystem:
    """Simulates a document management system to demonstrate IDOR vulnerabilities"""
    
    def __init__(self):
        # Sample documents in the system
        self.documents = {
            101: {"title": "Alice's Personal Notes", "owner_id": 1, "content": "My private thoughts..."},
            102: {"title": "Bob's Project Plan", "owner_id": 2, "content": "Secret project details..."},
            103: {"title": "Company Payroll Data", "owner_id": 1, "content": "Salary information..."},
            104: {"title": "HR Policies", "owner_id": 3, "content": "Employee handbook..."},
            105: {"title": "Financial Reports", "owner_id": 1, "content": "Quarterly earnings..."}
        }
        
        # User database
        self.users = {
            1: {"name": "Alice", "role": "admin"},
            2: {"name": "Bob", "role": "employee"},
            3: {"name": "Charlie", "role": "hr"}
        }
    
    def vulnerable_get_document(self, doc_id: int, user_id: int):
        """
        ⚠️ VULNERABLE: No authorization check for document access
        """
        print(f"\n🔓 VULNERABLE Document Access")
        print(f"User {user_id} requesting document {doc_id}")
        
        if doc_id in self.documents:
            doc = self.documents[doc_id]
            print(f"✅ Document found: {doc['title']}")
            print(f"📄 Content: {doc['content'][:50]}...")
            return doc
        else:
            print(f"❌ Document {doc_id} not found")
            return None
    
    def secure_get_document(self, doc_id: int, user_id: int):
        """
        ✅ SECURE: Proper authorization check before access
        """
        print(f"\n🔒 SECURE Document Access")
        print(f"User {user_id} requesting document {doc_id}")
        
        # Step 1: Check if document exists
        if doc_id not in self.documents:
            print(f"❌ Document {doc_id} not found")
            return None
        
        doc = self.documents[doc_id]
        user = self.users.get(user_id)
        
        if not user:
            print(f"❌ Invalid user {user_id}")
            return None
        
        # Step 2: Authorization check
        can_access = self._can_access_document(user_id, doc_id)
        
        if can_access:
            print(f"✅ Access granted: {doc['title']}")
            print(f"📄 Content: {doc['content'][:50]}...")
            return doc
        else:
            print(f"❌ Access denied: Insufficient permissions")
            return None
    
    def _can_access_document(self, user_id: int, doc_id: int) -> bool:
        """Business logic for document access control"""
        doc = self.documents[doc_id]
        user = self.users[user_id]
        
        # Rule 1: Users can access their own documents
        if doc["owner_id"] == user_id:
            print(f"  📝 Access granted: User owns document")
            return True
        
        # Rule 2: Admins can access all documents
        if user["role"] == "admin":
            print(f"  👑 Access granted: Admin privileges")
            return True
        
        # Rule 3: HR can access HR documents
        if user["role"] == "hr" and "HR" in doc["title"]:
            print(f"  🏢 Access granted: HR document access")
            return True
        
        print(f"  🚫 Access denied: No permission for this document")
        return False

# Demonstrate IDOR vulnerability
doc_system = DocumentSystem()

print("📁 INSECURE DIRECT OBJECT REFERENCE (IDOR) DEMONSTRATION")
print("=" * 65)

print("\nAvailable Documents:")
for doc_id, doc in doc_system.documents.items():
    owner_name = doc_system.users[doc["owner_id"]]["name"]
    print(f"  {doc_id}: {doc['title']} (owned by {owner_name})")

print("\nUsers in System:")
for user_id, user in doc_system.users.items():
    print(f"  {user_id}: {user['name']} ({user['role']})")

# Demonstrate vulnerable access
print(f"\n" + "="*50)
print("VULNERABLE SYSTEM - No Authorization Checks")
print("="*50)

# Bob (employee) accessing his own document - should work
doc_system.vulnerable_get_document(102, 2)

# Bob trying to access Alice's private notes - should be blocked but isn't!
doc_system.vulnerable_get_document(101, 2)

# Bob accessing company payroll - definitely should be blocked but isn't!
doc_system.vulnerable_get_document(103, 2)

# Demonstrate secure access
print(f"\n" + "="*50)
print("SECURE SYSTEM - Proper Authorization Checks")
print("="*50)

# Bob accessing his own document - should work
doc_system.secure_get_document(102, 2)

# Bob trying to access Alice's private notes - properly blocked
doc_system.secure_get_document(101, 2)

# Bob accessing company payroll - properly blocked
doc_system.secure_get_document(103, 2)

# Alice (admin) accessing any document - should work
doc_system.secure_get_document(102, 1)
doc_system.secure_get_document(103, 1)

print(f"\n💡 IDOR Prevention:")
print("  ✅ Always verify user can access specific objects")
print("  ✅ Check ownership and role-based permissions")
print("  ✅ Use object-level access controls")
print("  ✅ Never rely on 'security through obscurity'")

### 1.7 Privilege Separation and Defense in Depth

Let's demonstrate how privilege separation and multiple security layers work together.

In [ ]:
import subprocess
import tempfile
from pathlib import Path

class PrivilegeSeparationDemo:
    """Demonstrates privilege separation concepts"""
    
    def __init__(self):
        self.audit_log = []
    
    def unsafe_file_operation(self, filename: str, operation: str):
        """
        ⚠️ UNSAFE: All operations run with same high privileges
        """
        print(f"\n⚠️  UNSAFE FILE OPERATION")
        print(f"Operation: {operation}")
        print(f"File: {filename}")
        print(f"Running as: Current user (potentially root/admin)")
        
        # Simulate dangerous operation
        if operation == "delete":
            print(f"🚨 DANGEROUS: Deleting {filename} with full system privileges!")
            print(f"💥 If this were root, could delete critical system files!")
        
        return "operation_completed_unsafely"
    
    def safe_file_operation(self, filename: str, operation: str, user_id: int):
        """
        ✅ SAFE: Multi-layer security with privilege separation
        """
        print(f"\n🛡️  SAFE FILE OPERATION WITH PRIVILEGE SEPARATION")
        print(f"Operation: {operation}")
        print(f"File: {filename}")
        print(f"Requesting user: {user_id}")
        
        # Layer 1: Input validation
        if not self._validate_input(filename, operation):
            print("❌ Layer 1 FAILED: Invalid input")
            return None
        print("✅ Layer 1 PASSED: Input validation")
        
        # Layer 2: Authentication check
        if not self._check_authentication(user_id):
            print("❌ Layer 2 FAILED: Authentication required")
            return None
        print("✅ Layer 2 PASSED: User authenticated")
        
        # Layer 3: Authorization check
        if not self._check_authorization(user_id, operation, filename):
            print("❌ Layer 3 FAILED: Insufficient permissions")
            return None
        print("✅ Layer 3 PASSED: User authorized")
        
        # Layer 4: Privilege separation - delegate to limited subprocess
        try:
            result = self._execute_with_limited_privileges(filename, operation, user_id)
            print("✅ Layer 4 PASSED: Operation completed safely")
        except Exception as e:
            print(f"❌ Layer 4 FAILED: {e}")
            return None
        
        # Layer 5: Audit logging
        self._audit_log_operation(user_id, operation, filename, "SUCCESS")
        print("✅ Layer 5 COMPLETED: Operation logged")
        
        return result
    
    def _validate_input(self, filename: str, operation: str) -> bool:
        """Layer 1: Input validation"""
        # Check for path traversal
        if ".." in filename or filename.startswith("/"):
            return False
        
        # Check operation whitelist
        if operation not in ["read", "write", "delete"]:
            return False
        
        return True
    
    def _check_authentication(self, user_id: int) -> bool:
        """Layer 2: Authentication check"""
        # Simulate checking active session
        valid_users = [1, 2, 3]  # Authenticated user IDs
        return user_id in valid_users
    
    def _check_authorization(self, user_id: int, operation: str, filename: str) -> bool:
        """Layer 3: Authorization check"""
        # Simulate role-based permissions
        user_roles = {1: "admin", 2: "user", 3: "readonly"}
        role = user_roles.get(user_id, "none")
        
        if role == "admin":
            return True
        elif role == "user" and operation in ["read", "write"]:
            return True
        elif role == "readonly" and operation == "read":
            return True
        
        return False
    
    def _execute_with_limited_privileges(self, filename: str, operation: str, user_id: int):
        """Layer 4: Execute with minimal privileges"""
        print(f"   🔧 Delegating to limited-privilege subprocess...")
        print(f"   📁 Working directory: /tmp/safe_operations/")
        print(f"   👤 Running as: limited_user (not root/admin)")
        print(f"   🔒 Permissions: Read/write to specific directory only")
        
        # In a real system, this would:
        # 1. Create a subprocess with limited user account
        # 2. Use chroot jail or container
        # 3. Apply resource limits (CPU, memory, disk)
        # 4. Use SELinux/AppArmor policies
        
        # For demo, simulate the operation
        if operation == "delete":
            print(f"   🗑️  Safely deleting {filename} in restricted environment")
        elif operation == "read":
            print(f"   📖 Safely reading {filename} with limited permissions")
        elif operation == "write":
            print(f"   ✏️  Safely writing {filename} with limited permissions")
        
        return f"{operation}_completed_safely"
    
    def _audit_log_operation(self, user_id: int, operation: str, filename: str, result: str):
        """Layer 5: Audit logging"""
        log_entry = {
            "timestamp": datetime.now().isoformat(),
            "user_id": user_id,
            "operation": operation,
            "filename": filename,
            "result": result
        }
        self.audit_log.append(log_entry)
    
    def show_audit_log(self):
        """Display audit trail"""
        print(f"\n📋 Security Audit Log")
        print("-" * 40)
        for entry in self.audit_log:
            print(f"[{entry['timestamp'][:19]}] User {entry['user_id']}: "
                  f"{entry['operation']} {entry['filename']} -> {entry['result']}")

# Demonstrate privilege separation
priv_demo = PrivilegeSeparationDemo()

print("🔐 PRIVILEGE SEPARATION & DEFENSE IN DEPTH DEMONSTRATION")
print("=" * 65)

# Unsafe operations
print("\n" + "="*50)
print("UNSAFE APPROACH - Single Point of Failure")
print("="*50)

priv_demo.unsafe_file_operation("important_file.txt", "delete")
priv_demo.unsafe_file_operation("../../../etc/passwd", "delete")

# Safe operations
print("\n" + "="*50)
print("SAFE APPROACH - Multiple Security Layers")
print("="*50)

# Valid operation by admin
print(f"\n🔍 Test 1: Valid admin operation")
priv_demo.safe_file_operation("report.txt", "delete", 1)

# Invalid operation attempt
print(f"\n🔍 Test 2: Path traversal attack attempt")
priv_demo.safe_file_operation("../../../etc/passwd", "delete", 2)

# Unauthorized operation attempt
print(f"\n🔍 Test 3: Privilege escalation attempt")
priv_demo.safe_file_operation("sensitive.txt", "delete", 3)

# Valid read operation
print(f"\n🔍 Test 4: Valid read operation")
priv_demo.safe_file_operation("public.txt", "read", 3)

# Show audit log
priv_demo.show_audit_log()

print(f"\n🛡️  Defense in Depth Benefits:")
print("  ✅ Multiple independent security layers")
print("  ✅ Failure of one layer doesn't compromise system")
print("  ✅ Privilege separation limits blast radius")
print("  ✅ Comprehensive audit trail for investigation")
print("  ✅ Each layer can be maintained by different teams")

### 1.8 Authorization Security Monitoring and Alerting

Finally, let's implement a monitoring system that can detect authorization attacks and suspicious patterns.

In [ ]:
from collections import defaultdict, Counter
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

class AuthorizationMonitor:
    """Advanced authorization monitoring and threat detection"""
    
    def __init__(self):
        self.events = []
        self.user_patterns = defaultdict(list)
        self.alert_rules = {
            'rapid_denials': {'threshold': 5, 'window_minutes': 5},
            'privilege_escalation': {'threshold': 3, 'window_minutes': 10},
            'unusual_access': {'threshold': 10, 'window_minutes': 60}
        }
    
    def log_authorization_event(self, user_id: str, resource: str, action: str, 
                              result: str, risk_level: str = "LOW"):
        """Log authorization event with security context"""
        event = {
            'timestamp': datetime.now(),
            'user_id': user_id,
            'resource': resource,
            'action': action,
            'result': result,  # GRANTED, DENIED, ERROR
            'risk_level': risk_level,  # LOW, MEDIUM, HIGH, CRITICAL
            'source_ip': f"192.168.1.{hash(user_id) % 255}",  # Simulated
            'user_agent': 'Mozilla/5.0...',  # Simulated
        }
        
        self.events.append(event)
        self.user_patterns[user_id].append(event)
        
        # Real-time threat detection
        alerts = self._analyze_event_patterns(event)
        if alerts:
            self._trigger_alerts(alerts)
    
    def _analyze_event_patterns(self, event):
        """Analyze patterns for suspicious behavior"""
        alerts = []
        user_id = event['user_id']
        now = event['timestamp']
        
        # Get recent events for this user
        recent_events = [e for e in self.user_patterns[user_id] 
                        if (now - e['timestamp']).total_seconds() < 3600]  # Last hour
        
        # Pattern 1: Rapid access denials (potential brute force)
        denied_events = [e for e in recent_events[-10:] if e['result'] == 'DENIED']
        if len(denied_events) >= self.alert_rules['rapid_denials']['threshold']:
            alerts.append({
                'type': 'RAPID_DENIALS',
                'severity': 'HIGH',
                'description': f'User {user_id} had {len(denied_events)} denials in recent activity',
                'recommendation': 'Consider temporary account lockout'
            })
        
        # Pattern 2: Privilege escalation attempts
        high_privilege_denials = [e for e in recent_events 
                                if e['result'] == 'DENIED' and 
                                any(keyword in e['action'].lower() 
                                    for keyword in ['admin', 'delete', 'manage', 'system'])]
        
        if len(high_privilege_denials) >= self.alert_rules['privilege_escalation']['threshold']:
            alerts.append({
                'type': 'PRIVILEGE_ESCALATION',
                'severity': 'CRITICAL',
                'description': f'User {user_id} attempted {len(high_privilege_denials)} privilege escalations',
                'recommendation': 'Immediate security review required'
            })
        
        # Pattern 3: Unusual access patterns
        resource_count = len(set(e['resource'] for e in recent_events))
        if resource_count >= self.alert_rules['unusual_access']['threshold']:
            alerts.append({
                'type': 'UNUSUAL_ACCESS',
                'severity': 'MEDIUM',
                'description': f'User {user_id} accessed {resource_count} different resources',
                'recommendation': 'Review user activity for potential compromise'
            })
        
        return alerts
    
    def _trigger_alerts(self, alerts):
        """Process security alerts"""
        for alert in alerts:
            print(f"\n🚨 SECURITY ALERT - {alert['severity']}")
            print(f"Type: {alert['type']}")
            print(f"Description: {alert['description']}")
            print(f"Recommendation: {alert['recommendation']}")
            print(f"Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    def generate_security_report(self):
        """Generate comprehensive security analysis"""
        if not self.events:
            print("No authorization events to analyze")
            return
        
        print(f"\n📊 AUTHORIZATION SECURITY REPORT")
        print("=" * 50)
        print(f"Analysis Period: {self.events[0]['timestamp'].strftime('%Y-%m-%d %H:%M')} to "
              f"{self.events[-1]['timestamp'].strftime('%Y-%m-%d %H:%M')}")
        print(f"Total Events: {len(self.events)}")
        
        # Overall statistics
        results = Counter(e['result'] for e in self.events)
        risk_levels = Counter(e['risk_level'] for e in self.events)
        
        print(f"\n📈 Event Summary:")
        for result, count in results.items():
            percentage = (count / len(self.events)) * 100
            print(f"  {result}: {count} ({percentage:.1f}%)")
        
        print(f"\n⚠️  Risk Level Distribution:")
        for risk, count in risk_levels.items():
            percentage = (count / len(self.events)) * 100
            print(f"  {risk}: {count} ({percentage:.1f}%)")
        
        # User activity analysis
        user_activity = defaultdict(lambda: {'granted': 0, 'denied': 0})
        for event in self.events:
            if event['result'] == 'GRANTED':
                user_activity[event['user_id']]['granted'] += 1
            else:
                user_activity[event['user_id']]['denied'] += 1
        
        print(f"\n👥 User Activity Analysis:")
        for user_id, activity in user_activity.items():
            total = activity['granted'] + activity['denied']
            denial_rate = (activity['denied'] / total) * 100 if total > 0 else 0
            status = "🔴 HIGH RISK" if denial_rate > 50 else "🟡 MEDIUM RISK" if denial_rate > 20 else "🟢 NORMAL"
            print(f"  {user_id}: {activity['granted']} granted, {activity['denied']} denied "
                  f"({denial_rate:.1f}% denial rate) {status}")
        
        # Resource access patterns
        resource_access = Counter(e['resource'] for e in self.events)
        print(f"\n📁 Most Accessed Resources:")
        for resource, count in resource_access.most_common(5):
            print(f"  {resource}: {count} accesses")
    
    def visualize_security_trends(self):
        """Create visualizations of authorization patterns"""
        if len(self.events) < 2:
            print("Insufficient data for visualization")
            return
        
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
        
        # Plot 1: Authorization results over time
        timestamps = [e['timestamp'] for e in self.events]
        granted = [1 if e['result'] == 'GRANTED' else 0 for e in self.events]
        denied = [1 if e['result'] == 'DENIED' else 0 for e in self.events]
        
        ax1.plot(timestamps, np.cumsum(granted), label='Granted', color='green')
        ax1.plot(timestamps, np.cumsum(denied), label='Denied', color='red')
        ax1.set_title('Cumulative Authorization Results')
        ax1.set_xlabel('Time')
        ax1.set_ylabel('Count')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Plot 2: Risk level distribution
        risk_levels = [e['risk_level'] for e in self.events]
        risk_counts = Counter(risk_levels)
        colors = {'LOW': 'green', 'MEDIUM': 'yellow', 'HIGH': 'orange', 'CRITICAL': 'red'}
        
        ax2.bar(risk_counts.keys(), risk_counts.values(), 
                color=[colors.get(k, 'gray') for k in risk_counts.keys()])
        ax2.set_title('Events by Risk Level')
        ax2.set_xlabel('Risk Level')
        ax2.set_ylabel('Count')
        
        # Plot 3: User activity comparison
        user_activity = defaultdict(lambda: {'granted': 0, 'denied': 0})
        for event in self.events:
            if event['result'] == 'GRANTED':
                user_activity[event['user_id']]['granted'] += 1
            else:
                user_activity[event['user_id']]['denied'] += 1
        
        users = list(user_activity.keys())
        granted_counts = [user_activity[u]['granted'] for u in users]
        denied_counts = [user_activity[u]['denied'] for u in users]
        
        x = np.arange(len(users))
        width = 0.35
        
        ax3.bar(x - width/2, granted_counts, width, label='Granted', color='green', alpha=0.7)
        ax3.bar(x + width/2, denied_counts, width, label='Denied', color='red', alpha=0.7)
        ax3.set_title('User Authorization Activity')
        ax3.set_xlabel('Users')
        ax3.set_ylabel('Count')
        ax3.set_xticks(x)
        ax3.set_xticklabels(users)
        ax3.legend()
        
        # Plot 4: Hourly activity pattern
        hours = [e['timestamp'].hour for e in self.events]
        hour_counts = Counter(hours)
        
        all_hours = list(range(24))
        counts = [hour_counts.get(h, 0) for h in all_hours]
        
        ax4.bar(all_hours, counts, color='steelblue', alpha=0.7)
        ax4.set_title('Authorization Activity by Hour')
        ax4.set_xlabel('Hour of Day')
        ax4.set_ylabel('Count')
        ax4.set_xticks(range(0, 24, 4))
        
        plt.tight_layout()
        plt.show()

# Demonstrate authorization monitoring
monitor = AuthorizationMonitor()

print("🔍 AUTHORIZATION MONITORING & THREAT DETECTION")
print("=" * 55)

# Simulate various authorization events
print("\n📝 Simulating authorization events...")

# Normal user activity
events_to_simulate = [
    ("alice", "dashboard", "view", "GRANTED", "LOW"),
    ("alice", "profile", "edit", "GRANTED", "LOW"),
    ("bob", "reports", "view", "GRANTED", "LOW"),
    ("bob", "reports", "export", "GRANTED", "MEDIUM"),
    
    # Suspicious activity - rapid denials
    ("eve", "admin_panel", "access", "DENIED", "HIGH"),
    ("eve", "user_management", "access", "DENIED", "HIGH"),
    ("eve", "system_config", "access", "DENIED", "HIGH"),
    ("eve", "database", "access", "DENIED", "CRITICAL"),
    ("eve", "admin_panel", "access", "DENIED", "HIGH"),
    ("eve", "delete_users", "attempt", "DENIED", "CRITICAL"),
    
    # More normal activity
    ("charlie", "documents", "read", "GRANTED", "LOW"),
    ("diana", "calendar", "view", "GRANTED", "LOW"),
    
    # Unusual access patterns
    ("mallory", "file1", "read", "GRANTED", "LOW"),
    ("mallory", "file2", "read", "GRANTED", "LOW"),
    ("mallory", "file3", "read", "GRANTED", "LOW"),
    ("mallory", "file4", "read", "GRANTED", "LOW"),
    ("mallory", "file5", "read", "GRANTED", "LOW"),
    ("mallory", "file6", "read", "GRANTED", "LOW"),
    ("mallory", "file7", "read", "GRANTED", "LOW"),
    ("mallory", "file8", "read", "GRANTED", "LOW"),
    ("mallory", "file9", "read", "GRANTED", "LOW"),
    ("mallory", "file10", "read", "GRANTED", "LOW"),
    ("mallory", "file11", "read", "GRANTED", "LOW"),
]

# Log all events
for user_id, resource, action, result, risk in events_to_simulate:
    monitor.log_authorization_event(user_id, resource, action, result, risk)
    time.sleep(0.1)  # Small delay to simulate real time

# Generate security report
monitor.generate_security_report()

# Create visualizations
print(f"\n📊 Generating security trend visualizations...")
monitor.visualize_security_trends()

print(f"\n🛡️  Monitoring System Benefits:")
print("  ✅ Real-time threat detection")
print("  ✅ Pattern analysis and alerting")
print("  ✅ Comprehensive audit trails")
print("  ✅ Risk-based event classification")
print("  ✅ Automated incident response triggers")

### Section 1 Summary: Key Learnings from Authorization Demonstrations

**🎯 Core Authorization Principles:**
- **Authentication First**: Always verify identity before checking permissions
- **Server-Side Decisions**: Never trust authorization data from clients
- **Principle of Least Privilege**: Grant minimum required permissions
- **Defense in Depth**: Multiple independent security layers

**🚨 Common Vulnerabilities Demonstrated:**
- **User-Supplied Authorization Data**: Trusting cookies, URLs, or form fields
- **Missing Authentication Checks**: Authorization without verifying identity
- **Insecure Direct Object References**: No ownership verification
- **Insufficient Privilege Separation**: Single point of failure systems

**🛡️ Secure Implementation Patterns:**
- **Role-Based Access Control (RBAC)**: Scalable permission management
- **Object-Level Authorization**: Verify access to specific resources
- **Privilege Separation**: Minimize blast radius of security failures
- **Comprehensive Monitoring**: Real-time threat detection and response

### Practical Applications

These demonstrations show real-world security concepts:

1. **Secure Web Applications**: Implement proper authorization checks at every endpoint
2. **API Security**: Protect REST/GraphQL APIs from IDOR and privilege escalation
3. **System Administration**: Use principle of least privilege for service accounts
4. **Security Monitoring**: Deploy automated threat detection systems

### Next Steps for Implementation

- Apply RBAC patterns in your development projects
- Implement comprehensive authorization logging
- Practice security testing with different user roles
- Study OAuth, SAML, and other enterprise authorization standards

The hands-on demonstrations reinforce that authorization is not just about coding - it's about designing secure systems that fail safely and provide comprehensive protection against real-world attacks.

In [ ]:
# Section 1 Complete - Authorization Security Demonstrations
print("🎉 Chapter 4 Section 1 - COMPLETE!")
print("=" * 50)
print("Authorization Demonstrations Completed:")
print("✅ Authentication vs Authorization distinctions")
print("✅ Secure authorization system implementation")
print("✅ Vulnerability demonstrations (user-supplied data)")
print("✅ IDOR (Insecure Direct Object Reference) examples")
print("✅ Privilege separation and defense in depth")
print("✅ Security monitoring and threat detection")
print()
print("🔧 You now have hands-on experience with:")
print("   • Role-Based Access Control (RBAC) design")
print("   • Common authorization attack vectors")
print("   • Secure coding patterns for authorization")
print("   • Multi-layer security architecture")
print("   • Real-time security monitoring systems")
print()
print("🚀 Ready for Section 2: Interactive Exercise Solutions!")
print("   Next: Hands-on exercises for authorization design and implementation")

---
## Section 2: Interactive Exercise Solutions

Now let's work through the Chapter 4 exercises interactively, implementing comprehensive solutions that demonstrate real-world authorization design patterns.

### Exercise 1: Authorization Activity Analysis

**Exercise Goal**: Think about a software system you use that involves users with different privileges. Draft a list of activities that should involve authorization.

**What you'll do**:
1. Analyze common software systems and their authorization requirements
2. Create comprehensive authorization matrices for different user roles
3. Identify critical authorization checkpoints
4. Design role-based permission systems

In [ ]:
# Exercise 1: Interactive Authorization Activity Analysis
import pandas as pd
from tabulate import tabulate

class AuthorizationAnalyzer:
    """Interactive system for analyzing authorization requirements across different software systems"""
    
    def __init__(self):
        # Define common software systems and their characteristics
        self.software_systems = {
            "email_system": {
                "name": "Corporate Email System",
                "description": "Enterprise email with shared mailboxes and admin functions",
                "user_roles": ["end_user", "manager", "it_admin", "compliance_officer"],
                "resources": ["own_mailbox", "shared_mailbox", "user_accounts", "system_settings", "audit_logs"],
                "actions": ["read", "send", "delete", "archive", "create_user", "reset_password", "view_logs", "export_data"]
            },
            
            "banking_app": {
                "name": "Online Banking Application",
                "description": "Customer banking with different account types and admin functions",
                "user_roles": ["customer", "premium_customer", "teller", "manager", "auditor"],
                "resources": ["own_accounts", "customer_accounts", "transactions", "reports", "system_config"],
                "actions": ["view_balance", "transfer", "pay_bills", "view_statements", "create_account", "freeze_account", "generate_reports"]
            },
            
            "hospital_system": {
                "name": "Hospital Management System",
                "description": "Electronic health records with HIPAA compliance requirements",
                "user_roles": ["patient", "nurse", "doctor", "administrator", "billing"],
                "resources": ["own_records", "patient_records", "schedules", "billing_info", "system_logs"],
                "actions": ["view_records", "update_records", "schedule_appointment", "prescribe", "bill_patient", "access_emergency_records"]
            },
            
            "ecommerce_platform": {
                "name": "E-commerce Platform",
                "description": "Online store with customer accounts and seller management",
                "user_roles": ["customer", "seller", "customer_service", "admin", "analyst"],
                "resources": ["own_profile", "orders", "products", "seller_accounts", "analytics"],
                "actions": ["browse", "purchase", "review", "list_product", "view_orders", "refund", "ban_user", "view_analytics"]
            },
            
            "university_system": {
                "name": "University Student Information System",
                "description": "Academic records, course management, and administrative functions",
                "user_roles": ["student", "instructor", "advisor", "registrar", "dean"],
                "resources": ["own_records", "student_records", "courses", "grades", "transcripts"],
                "actions": ["view_grades", "register_course", "submit_grade", "view_transcript", "modify_records", "generate_reports"]
            }
        }
    
    def analyze_system(self, system_key: str):
        """Analyze authorization requirements for a specific system"""
        if system_key not in self.software_systems:
            print(f"System '{system_key}' not found. Available systems: {list(self.software_systems.keys())}")
            return
        
        system = self.software_systems[system_key]
        
        print(f"🏢 AUTHORIZATION ANALYSIS: {system['name']}")
        print("=" * 60)
        print(f"Description: {system['description']}")
        print()
        
        # Create authorization matrix
        authorization_matrix = self._create_authorization_matrix(system)
        
        # Display role descriptions
        print("👥 USER ROLES:")
        role_descriptions = self._get_role_descriptions(system_key)
        for role, description in role_descriptions.items():
            print(f"  • {role.replace('_', ' ').title()}: {description}")
        
        print(f"\n🔐 AUTHORIZATION MATRIX:")
        print("-" * 40)
        
        # Create and display the matrix
        df = pd.DataFrame(authorization_matrix)
        df = df.fillna('❌')  # Fill NaN with deny symbol
        
        # Convert to readable format
        print(tabulate(df, headers='keys', tablefmt='grid'))
        
        # Analyze critical authorization points
        self._analyze_critical_points(system, authorization_matrix)
        
        # Security recommendations
        self._generate_security_recommendations(system_key, system)
        
        return authorization_matrix
    
    def _create_authorization_matrix(self, system):
        """Create authorization matrix showing which roles can perform which actions on which resources"""
        roles = system["user_roles"]
        resources = system["resources"]
        actions = system["actions"]
        
        # Define role hierarchies and permissions (business logic)
        permissions = self._define_permissions(system)
        
        # Create matrix with action-resource combinations
        matrix_data = {}
        
        for role in roles:
            role_permissions = []
            for action in actions:
                for resource in resources:
                    permission_key = f"{action}_{resource}"
                    if role in permissions and permission_key in permissions[role]:
                        permission_result = permissions[role][permission_key]
                        if permission_result:
                            role_permissions.append(f"✅ {action} {resource}")
                        else:
                            role_permissions.append(f"❌ {action} {resource}")
                    else:
                        # Default deny for undefined permissions
                        role_permissions.append(f"❌ {action} {resource}")
            
            matrix_data[role.replace('_', ' ').title()] = role_permissions[:10]  # Limit for display
        
        return matrix_data
    
    def _define_permissions(self, system):
        """Define specific permissions for each role based on business logic"""
        system_name = system["name"].lower()
        
        if "email" in system_name:
            return self._email_permissions()
        elif "banking" in system_name:
            return self._banking_permissions()
        elif "hospital" in system_name:
            return self._hospital_permissions()
        elif "ecommerce" in system_name:
            return self._ecommerce_permissions()
        elif "university" in system_name:
            return self._university_permissions()
        else:
            return {}
    
    def _email_permissions(self):
        """Define email system permissions"""
        return {
            "end_user": {
                "read_own_mailbox": True, "send_own_mailbox": True, "delete_own_mailbox": True,
                "read_shared_mailbox": False, "create_user_user_accounts": False,
                "view_logs_audit_logs": False, "export_data_audit_logs": False
            },
            "manager": {
                "read_own_mailbox": True, "send_own_mailbox": True, "delete_own_mailbox": True,
                "read_shared_mailbox": True, "send_shared_mailbox": True,
                "view_logs_audit_logs": True, "export_data_audit_logs": False
            },
            "it_admin": {
                "read_own_mailbox": True, "create_user_user_accounts": True,
                "reset_password_user_accounts": True, "view_logs_audit_logs": True,
                "export_data_audit_logs": True, "read_system_settings": True
            },
            "compliance_officer": {
                "read_own_mailbox": True, "view_logs_audit_logs": True,
                "export_data_audit_logs": True, "archive_shared_mailbox": True
            }
        }
    
    def _banking_permissions(self):
        """Define banking system permissions"""
        return {
            "customer": {
                "view_balance_own_accounts": True, "transfer_own_accounts": True,
                "pay_bills_own_accounts": True, "view_statements_own_accounts": True,
                "view_balance_customer_accounts": False, "create_account_customer_accounts": False
            },
            "premium_customer": {
                "view_balance_own_accounts": True, "transfer_own_accounts": True,
                "pay_bills_own_accounts": True, "view_statements_own_accounts": True,
                "generate_reports_own_accounts": True, "view_balance_customer_accounts": False
            },
            "teller": {
                "view_balance_customer_accounts": True, "create_account_customer_accounts": True,
                "freeze_account_customer_accounts": True, "view_statements_customer_accounts": True,
                "transfer_customer_accounts": True, "generate_reports_reports": False
            },
            "manager": {
                "view_balance_customer_accounts": True, "create_account_customer_accounts": True,
                "freeze_account_customer_accounts": True, "generate_reports_reports": True,
                "view_statements_customer_accounts": True, "read_system_config": True
            },
            "auditor": {
                "view_statements_customer_accounts": True, "generate_reports_reports": True,
                "view_logs_system_config": True, "export_data_reports": True,
                "transfer_customer_accounts": False, "create_account_customer_accounts": False
            }
        }
    
    def _hospital_permissions(self):
        """Define hospital system permissions with HIPAA considerations"""
        return {
            "patient": {
                "view_records_own_records": True, "update_records_own_records": False,
                "schedule_appointment_schedules": True, "view_records_patient_records": False,
                "access_emergency_records_patient_records": False
            },
            "nurse": {
                "view_records_patient_records": True, "update_records_patient_records": True,
                "schedule_appointment_schedules": True, "view_records_own_records": True,
                "prescribe_patient_records": False, "bill_patient_billing_info": False
            },
            "doctor": {
                "view_records_patient_records": True, "update_records_patient_records": True,
                "prescribe_patient_records": True, "access_emergency_records_patient_records": True,
                "schedule_appointment_schedules": True, "view_records_own_records": True
            },
            "administrator": {
                "view_records_patient_records": False, "update_records_patient_records": False,
                "schedule_appointment_schedules": True, "view_logs_system_logs": True,
                "bill_patient_billing_info": True, "generate_reports_system_logs": True
            },
            "billing": {
                "bill_patient_billing_info": True, "view_statements_billing_info": True,
                "view_records_patient_records": False, "update_records_patient_records": False,
                "generate_reports_billing_info": True
            }
        }
    
    def _ecommerce_permissions(self):
        """Define e-commerce platform permissions"""
        return {
            "customer": {
                "browse_products": True, "purchase_products": True, "review_products": True,
                "view_orders_own_profile": True, "refund_orders": False, "ban_user_seller_accounts": False
            },
            "seller": {
                "browse_products": True, "list_product_products": True, "view_orders_orders": True,
                "view_analytics_own_profile": True, "ban_user_seller_accounts": False, "refund_orders": False
            },
            "customer_service": {
                "view_orders_orders": True, "refund_orders": True, "view_orders_customer_accounts": True,
                "ban_user_seller_accounts": False, "view_analytics_analytics": False, "list_product_products": False
            },
            "admin": {
                "ban_user_seller_accounts": True, "view_analytics_analytics": True,
                "view_orders_orders": True, "refund_orders": True, "list_product_products": True,
                "browse_products": True
            },
            "analyst": {
                "view_analytics_analytics": True, "browse_products": True,
                "view_orders_orders": True, "ban_user_seller_accounts": False, "refund_orders": False
            }
        }
    
    def _university_permissions(self):
        """Define university system permissions"""
        return {
            "student": {
                "view_grades_own_records": True, "register_course_courses": True,
                "view_transcript_own_records": True, "submit_grade_grades": False,
                "modify_records_student_records": False, "generate_reports_transcripts": False
            },
            "instructor": {
                "submit_grade_grades": True, "view_grades_courses": True,
                "view_records_student_records": True, "register_course_courses": False,
                "modify_records_student_records": False, "generate_reports_grades": True
            },
            "advisor": {
                "view_grades_student_records": True, "view_transcript_student_records": True,
                "register_course_courses": True, "submit_grade_grades": False,
                "modify_records_student_records": True, "generate_reports_student_records": True
            },
            "registrar": {
                "modify_records_student_records": True, "generate_reports_transcripts": True,
                "view_transcript_student_records": True, "register_course_courses": True,
                "submit_grade_grades": False, "view_grades_student_records": True
            },
            "dean": {
                "generate_reports_transcripts": True, "view_grades_student_records": True,
                "modify_records_student_records": True, "view_transcript_student_records": True,
                "submit_grade_grades": False, "register_course_courses": False
            }
        }
    
    def _get_role_descriptions(self, system_key):
        """Get detailed descriptions for each role"""
        descriptions = {
            "email_system": {
                "end_user": "Regular employee with basic email access",
                "manager": "Department manager with team oversight",
                "it_admin": "IT administrator with system management rights",
                "compliance_officer": "Legal/compliance with audit access"
            },
            "banking_app": {
                "customer": "Regular banking customer",
                "premium_customer": "High-value customer with additional features",
                "teller": "Bank teller with customer service access",
                "manager": "Branch manager with approval authority",
                "auditor": "External auditor with read-only access"
            },
            "hospital_system": {
                "patient": "Hospital patient with limited self-access",
                "nurse": "Licensed nurse with patient care access",
                "doctor": "Licensed physician with full patient access",
                "administrator": "Hospital administrator with system access",
                "billing": "Billing department with financial access"
            },
            "ecommerce_platform": {
                "customer": "Shopping customer with purchase rights",
                "seller": "Merchant with product management rights",
                "customer_service": "Support agent with order management",
                "admin": "Platform administrator with full access",
                "analyst": "Business analyst with reporting access"
            },
            "university_system": {
                "student": "Enrolled student with academic access",
                "instructor": "Faculty member with teaching rights",
                "advisor": "Academic advisor with student guidance rights",
                "registrar": "Registrar with academic record management",
                "dean": "Academic dean with departmental oversight"
            }
        }
        return descriptions.get(system_key, {})
    
    def _analyze_critical_points(self, system, matrix):
        """Identify critical authorization checkpoints"""
        print(f"\n🎯 CRITICAL AUTHORIZATION CHECKPOINTS:")
        print("-" * 45)
        
        critical_actions = {
            "email_system": ["create_user", "export_data", "view_logs"],
            "banking_app": ["freeze_account", "generate_reports", "transfer"],
            "hospital_system": ["access_emergency_records", "prescribe", "bill_patient"],
            "ecommerce_platform": ["ban_user", "refund", "view_analytics"],
            "university_system": ["modify_records", "submit_grade", "generate_reports"]
        }
        
        system_name = system["name"].lower()
        for sys_key, actions in critical_actions.items():
            if sys_key.split('_')[0] in system_name:
                print(f"High-risk actions requiring strong authorization:")
                for action in actions:
                    print(f"  🔴 {action.replace('_', ' ').title()}: Requires multi-factor authentication")
                break
        
        print(f"\nPrivilege escalation risks:")
        print(f"  ⚠️  Role transitions must be carefully controlled")
        print(f"  ⚠️  Temporary privilege elevation needs audit trails")
        print(f"  ⚠️  Emergency access procedures require approval")
    
    def _generate_security_recommendations(self, system_key, system):
        """Generate security recommendations for the system"""
        print(f"\n🛡️  SECURITY RECOMMENDATIONS:")
        print("-" * 35)
        
        general_recommendations = [
            "Implement principle of least privilege",
            "Use role-based access control (RBAC)",
            "Require multi-factor authentication for admin actions",
            "Maintain comprehensive audit logs",
            "Regular access reviews and role cleanup",
            "Implement time-based access restrictions",
            "Use approval workflows for sensitive operations"
        ]
        
        # System-specific recommendations
        specific_recommendations = {
            "email_system": [
                "Encrypt sensitive email content",
                "Implement data loss prevention (DLP)",
                "Regular mailbox access reviews"
            ],
            "banking_app": [
                "Implement transaction limits per role",
                "Real-time fraud detection",
                "Regulatory compliance monitoring"
            ],
            "hospital_system": [
                "HIPAA compliance auditing",
                "Break-glass emergency access procedures",
                "Patient consent tracking"
            ],
            "ecommerce_platform": [
                "Implement purchase approval workflows",
                "Monitor for account takeover attempts",
                "Seller verification processes"
            ],
            "university_system": [
                "FERPA compliance controls",
                "Academic integrity monitoring",
                "Grade change approval workflows"
            ]
        }
        
        print("General recommendations:")
        for rec in general_recommendations[:5]:  # Show top 5
            print(f"  ✅ {rec}")
        
        # Show system-specific recommendations
        for sys_key, recs in specific_recommendations.items():
            if sys_key.split('_')[0] in system["name"].lower():
                print(f"\nSystem-specific recommendations:")
                for rec in recs:
                    print(f"  🎯 {rec}")
                break

# Interactive Exercise 1 Implementation
print("🔐 EXERCISE 1: Interactive Authorization Activity Analysis")
print("=" * 60)

analyzer = AuthorizationAnalyzer()

print("Available software systems for analysis:")
for key, system in analyzer.software_systems.items():
    print(f"  • {key}: {system['name']}")

print(f"\n" + "="*70)
print("ANALYZING: Corporate Email System")
print("="*70)
analyzer.analyze_system("email_system")

print(f"\n" + "="*70)
print("ANALYZING: Online Banking Application")
print("="*70)
analyzer.analyze_system("banking_app")

### Exercise 2: Secure Syslog Management Web Service

**Exercise Goal**: Design a web service that allows a user to delete old syslog files from the system the web service runs on. How should authorization be handled? What components are required?

**What you'll do**:
1. Design a complete secure web service architecture
2. Implement multi-layer authorization for file system operations
3. Create privilege separation and audit mechanisms
4. Test security controls against common attack vectors

In [2]:
# Exercise 2: Secure Syslog Management Web Service
import os
import re
import subprocess
import tempfile
import secrets
from datetime import datetime, timedelta
from pathlib import Path
from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple
import json

@dataclass
class SyslogFile:
    """Represents a syslog file with metadata"""
    filename: str
    path: str
    size_bytes: int
    modified_date: datetime
    age_days: int
    file_type: str  # 'current', 'archived', 'rotated'

class SecureSyslogService:
    """
    Comprehensive secure web service for syslog file management
    Demonstrates proper authorization, privilege separation, and security controls
    """
    
    def __init__(self):
        # Configuration
        demo_logs_path = str(Path.cwd() / 'demo_logs')
        self.config = {
            'syslog_directories': ['/var/log', '/var/log/archived', demo_logs_path],
            'allowed_file_patterns': [r'.*\.log$', r'.*\.log\.\d+$', r'.*\.log\.gz$'],
            'min_age_days': 30,  # Minimum age before deletion allowed
            'max_file_size_mb': 1000,  # Maximum single file size to handle
            'service_user': 'syslog_service',  # Limited privilege user
            'audit_log_path': '/var/log/syslog_service_audit.log'
        }
        
        # User roles and permissions
        self.role_permissions = {
            'syslog_admin': {
                'list_files', 'delete_old_files', 'delete_any_file', 
                'view_audit_logs', 'configure_service'
            },
            'system_admin': {
                'list_files', 'delete_old_files', 'view_audit_logs'
            },
            'log_analyst': {
                'list_files', 'view_audit_logs'
            },
            'readonly_user': {
                'list_files'
            }
        }
        
        # Active sessions (in production, use Redis or database)
        self.active_sessions = {}
        
        # Audit trail
        self.audit_events = []
        
        # Create demo environment
        self._setup_demo_environment()
    
    def _setup_demo_environment(self):
        """Create demo syslog files for testing"""
        demo_dir = Path(self.config['syslog_directories'][2])
        demo_dir.mkdir(exist_ok=True)
        
        # Create sample log files with different ages
        sample_files = [
            ('current.log', 0),      # Today
            ('app.log.1', 5),        # 5 days old
            ('system.log.gz', 45),   # 45 days old (eligible for deletion)
            ('archive.log.2', 60),   # 60 days old (eligible for deletion)
            ('critical.log', 90),    # 90 days old (definitely eligible)
        ]
        
        for filename, days_old in sample_files:
            file_path = demo_dir / filename
            file_path.write_text(f"Demo log content for {filename}\n" * 100)
            
            # Modify timestamp to simulate age
            old_time = datetime.now() - timedelta(days=days_old)
            os.utime(file_path, (old_time.timestamp(), old_time.timestamp()))
        
        print(f"✅ Demo environment created with {len(sample_files)} sample log files")
    
    def authenticate_user(self, username: str, password: str, role: str) -> Optional[str]:
        """
        Secure user authentication
        Returns session token if successful
        """
        # Simulate authentication (in production, use proper auth system)
        valid_users = {
            'admin': {'password': 'admin_pass', 'role': 'syslog_admin'},
            'sysadmin': {'password': 'sys_pass', 'role': 'system_admin'},
            'analyst': {'password': 'analyst_pass', 'role': 'log_analyst'},
            'viewer': {'password': 'view_pass', 'role': 'readonly_user'}
        }
        
        user_data = valid_users.get(username)
        if user_data and user_data['password'] == password:
            # Create secure session
            session_token = secrets.token_urlsafe(32)
            self.active_sessions[session_token] = {
                'username': username,
                'role': user_data['role'],
                'login_time': datetime.now(),
                'last_activity': datetime.now()
            }
            
            self._audit_log('AUTHENTICATION', username, 'login_success', 
                          {'role': user_data['role']})
            
            print(f"✅ Authentication successful for {username} ({user_data['role']})")
            return session_token
        
        self._audit_log('AUTHENTICATION', username, 'login_failed', {'reason': 'invalid_credentials'})
        print(f"❌ Authentication failed for {username}")
        return None
    
    def authorize_action(self, session_token: str, required_permission: str) -> Tuple[bool, str]:
        """
        Multi-layer authorization check
        """
        # Layer 1: Validate session
        if session_token not in self.active_sessions:
            return False, "Invalid or expired session"
        
        session = self.active_sessions[session_token]
        username = session['username']
        role = session['role']
        
        # Layer 2: Check session expiry
        if (datetime.now() - session['last_activity']).total_seconds() > 3600:  # 1 hour
            del self.active_sessions[session_token]
            return False, "Session expired"
        
        # Layer 3: Role-based permission check
        user_permissions = self.role_permissions.get(role, set())
        if required_permission not in user_permissions:
            self._audit_log('AUTHORIZATION', username, 'access_denied', 
                          {'permission': required_permission, 'role': role})
            return False, f"Insufficient permissions for {required_permission}"
        
        # Update session activity
        session['last_activity'] = datetime.now()
        
        self._audit_log('AUTHORIZATION', username, 'access_granted', 
                      {'permission': required_permission, 'role': role})
        return True, "Authorized"
    
    def list_syslog_files(self, session_token: str, directory: str = None) -> List[SyslogFile]:
        """
        Securely list syslog files with authorization
        """
        # Authorization check
        authorized, reason = self.authorize_action(session_token, 'list_files')
        if not authorized:
            raise PermissionError(reason)
        
        username = self.active_sessions[session_token]['username']
        
        # Input validation
        if directory:
            if directory not in self.config['syslog_directories']:
                raise ValueError(f"Directory not in allowed list: {directory}")
        else:
            directory = self.config['syslog_directories'][2]  # Use demo directory
        
        # Secure file discovery
        syslog_files = []
        try:
            dir_path = Path(directory)
            if not dir_path.exists():
                self._audit_log('FILE_OPERATION', username, 'list_files_failed', 
                              {'directory': directory, 'reason': 'directory_not_found'})
                return []
            
            for file_path in dir_path.iterdir():
                if file_path.is_file() and self._is_valid_syslog_file(file_path.name):
                    stat = file_path.stat()
                    modified_date = datetime.fromtimestamp(stat.st_mtime)
                    age_days = (datetime.now() - modified_date).days
                    
                    syslog_file = SyslogFile(
                        filename=file_path.name,
                        path=str(file_path),
                        size_bytes=stat.st_size,
                        modified_date=modified_date,
                        age_days=age_days,
                        file_type=self._classify_file_type(file_path.name)
                    )
                    syslog_files.append(syslog_file)
            
            self._audit_log('FILE_OPERATION', username, 'list_files_success', 
                          {'directory': directory, 'file_count': len(syslog_files)})
            
        except Exception as e:
            self._audit_log('FILE_OPERATION', username, 'list_files_error', 
                          {'directory': directory, 'error': str(e)})
            raise
        
        return sorted(syslog_files, key=lambda x: x.modified_date, reverse=True)
    
    def delete_old_syslog_files(self, session_token: str, min_age_days: int = None, 
                               dry_run: bool = True) -> Dict[str, List[str]]:
        """
        Securely delete old syslog files with comprehensive authorization
        """
        # Determine required permission based on age
        if min_age_days and min_age_days < self.config['min_age_days']:
            required_permission = 'delete_any_file'
        else:
            required_permission = 'delete_old_files'
        
        # Authorization check
        authorized, reason = self.authorize_action(session_token, required_permission)
        if not authorized:
            raise PermissionError(reason)
        
        username = self.active_sessions[session_token]['username']
        min_age = min_age_days or self.config['min_age_days']
        
        print(f"🗑️  SYSLOG DELETION OPERATION")
        print(f"User: {username}")
        print(f"Minimum age: {min_age} days")
        print(f"Mode: {'DRY RUN' if dry_run else 'LIVE DELETION'}")
        print("-" * 40)
        
        results = {
            'deleted': [],
            'skipped': [],
            'errors': []
        }
        
        # Get files from demo directory only
        try:
            files = self.list_syslog_files(session_token, self.config['syslog_directories'][2])
            
            for syslog_file in files:
                try:
                    if self._should_delete_file(syslog_file, min_age):
                        if dry_run:
                            results['deleted'].append(f"[DRY RUN] {syslog_file.path}")
                            print(f"  🗑️  Would delete: {syslog_file.filename} ({syslog_file.age_days} days old)")
                        else:
                            # Actual deletion with privilege separation
                            success = self._secure_delete_file(syslog_file.path, username)
                            if success:
                                results['deleted'].append(syslog_file.path)
                                print(f"  ✅ Deleted: {syslog_file.filename}")
                            else:
                                results['errors'].append(f"Failed to delete: {syslog_file.path}")
                                print(f"  ❌ Failed: {syslog_file.filename}")
                    else:
                        results['skipped'].append(syslog_file.path)
                        print(f"  ⏭️  Skipped: {syslog_file.filename} ({syslog_file.age_days} days old - too recent)")
                
                except Exception as e:
                    error_msg = f"Error processing {syslog_file.filename}: {str(e)}"
                    results['errors'].append(error_msg)
                    print(f"  ❌ Error: {error_msg}")
                    
        except Exception as e:
            error_msg = f"Error accessing files: {str(e)}"
            results['errors'].append(error_msg)
            print(f"❌ Error: {error_msg}")
        
        # Audit the operation
        self._audit_log('FILE_DELETION', username, 'bulk_delete_operation', {
            'min_age_days': min_age,
            'dry_run': dry_run,
            'deleted_count': len(results['deleted']),
            'skipped_count': len(results['skipped']),
            'error_count': len(results['errors'])
        })
        
        return results
    
    def _secure_delete_file(self, file_path: str, username: str) -> bool:
        """
        Securely delete file using privilege separation
        """
        try:
            # Layer 1: Final path validation
            if not self._is_safe_path(file_path):
                raise ValueError("Unsafe file path detected")
            
            # Layer 2: File existence and permission check
            path_obj = Path(file_path)
            if not path_obj.exists():
                raise FileNotFoundError(f"File not found: {file_path}")
            
            # Layer 3: Execute deletion with limited privileges
            result = self._execute_with_limited_privileges(
                'delete_file', 
                {'file_path': file_path, 'user': username}
            )
            
            if result['success']:
                self._audit_log('FILE_DELETION', username, 'file_deleted', 
                              {'file_path': file_path, 'method': 'secure_delete'})
                return True
            else:
                self._audit_log('FILE_DELETION', username, 'file_delete_failed', 
                              {'file_path': file_path, 'reason': result['error']})
                return False
                
        except Exception as e:
            self._audit_log('FILE_DELETION', username, 'file_delete_error', 
                          {'file_path': file_path, 'error': str(e)})
            return False
    
    def _execute_with_limited_privileges(self, operation: str, params: dict) -> dict:
        """
        Simulate execution with limited privileges (privilege separation)
        """
        print(f"    🔧 Executing '{operation}' with limited privileges...")
        print(f"    👤 Running as: {self.config['service_user']} (not root)")
        print(f"    🔒 Restricted to: {self.config['syslog_directories']}")
        
        # Simulate the operation
        if operation == 'delete_file':
            file_path = params['file_path']
            try:
                # In production, this would be a subprocess call:
                # subprocess.run(['sudo', '-u', 'syslog_service', 'rm', file_path])
                
                # For demo, actually delete the file
                os.remove(file_path)
                return {'success': True}
            except Exception as e:
                return {'success': False, 'error': str(e)}
        
        return {'success': False, 'error': 'Unknown operation'}
    
    def _is_valid_syslog_file(self, filename: str) -> bool:
        """Validate if file matches allowed syslog patterns"""
        for pattern in self.config['allowed_file_patterns']:
            if re.match(pattern, filename):
                return True
        return False
    
    def _is_safe_path(self, file_path: str) -> bool:
        """Validate file path for security"""
        # For demo, allow demo_logs files
        demo_dir = self.config['syslog_directories'][2]
        if demo_dir in file_path:
            return True
            
        # Prevent path traversal
        if '..' in file_path:
            return False
        
        # Ensure path is in allowed directories
        path_obj = Path(file_path).resolve()
        for allowed_dir in self.config['syslog_directories']:
            if str(path_obj).startswith(str(Path(allowed_dir).resolve())):
                return True
        
        return False
    
    def _classify_file_type(self, filename: str) -> str:
        """Classify log file type"""
        if filename.endswith('.gz'):
            return 'archived'
        elif re.search(r'\.\d+$', filename):
            return 'rotated'
        else:
            return 'current'
    
    def _should_delete_file(self, syslog_file: SyslogFile, min_age_days: int) -> bool:
        """Business logic for file deletion eligibility"""
        # Age check
        if syslog_file.age_days < min_age_days:
            return False
        
        # Never delete current active logs
        if syslog_file.file_type == 'current' and syslog_file.age_days < 1:
            return False
        
        # Size check for safety
        if syslog_file.size_bytes > self.config['max_file_size_mb'] * 1024 * 1024:
            return False
        
        return True
    
    def _audit_log(self, category: str, user: str, action: str, details: dict):
        """Comprehensive audit logging"""
        event = {
            'timestamp': datetime.now().isoformat(),
            'category': category,
            'user': user,
            'action': action,
            'details': details,
            'source_ip': '192.168.1.100'  # In production, get from request
        }
        
        self.audit_events.append(event)
        
        # In production, write to secure audit log file
        print(f"📝 AUDIT: [{category}] {user} -> {action}")
    
    def get_audit_logs(self, session_token: str, hours: int = 24) -> List[dict]:
        """Retrieve audit logs with authorization"""
        authorized, reason = self.authorize_action(session_token, 'view_audit_logs')
        if not authorized:
            raise PermissionError(reason)
        
        # Filter recent events
        cutoff_time = datetime.now() - timedelta(hours=hours)
        recent_events = [
            event for event in self.audit_events 
            if datetime.fromisoformat(event['timestamp']) > cutoff_time
        ]
        
        return recent_events
    
    def generate_security_report(self, session_token: str) -> dict:
        """Generate comprehensive security report"""
        authorized, reason = self.authorize_action(session_token, 'view_audit_logs')
        if not authorized:
            raise PermissionError(reason)
        
        username = self.active_sessions[session_token]['username']
        
        # Analyze audit events
        total_events = len(self.audit_events)
        categories = {}
        users = {}
        
        for event in self.audit_events:
            category = event['category']
            user = event['user']
            
            categories[category] = categories.get(category, 0) + 1
            users[user] = users.get(user, 0) + 1
        
        report = {
            'generated_by': username,
            'generated_at': datetime.now().isoformat(),
            'total_events': total_events,
            'event_categories': categories,
            'user_activity': users,
            'active_sessions': len(self.active_sessions),
            'security_recommendations': [
                'Regular audit log review',
                'Implement automated alerting for suspicious patterns',
                'Regular access review and role cleanup',
                'Monitor for privilege escalation attempts'
            ]
        }
        
        self._audit_log('SECURITY', username, 'report_generated', 
                      {'report_type': 'security_summary'})
        
        return report

# Interactive Exercise 2 Implementation
print("🔐 EXERCISE 2: Secure Syslog Management Web Service")
print("=" * 60)

# Initialize the service
syslog_service = SecureSyslogService()

print(f"\n🔐 Service Components:")
print("  ✅ Multi-layer authorization system")
print("  ✅ Role-based access control (RBAC)")
print("  ✅ Privilege separation architecture")
print("  ✅ Comprehensive audit logging")
print("  ✅ Secure file operations")
print("  ✅ Input validation and path traversal protection")

print(f"\n👥 Available User Roles:")
for role, permissions in syslog_service.role_permissions.items():
    print(f"  • {role}: {', '.join(permissions)}")

# Demonstrate authentication and authorization
print(f"\n" + "="*50)
print("DEMONSTRATION: User Authentication & Authorization")
print("="*50)

# Test different user types
test_users = [
    ('admin', 'admin_pass', 'syslog_admin'),
    ('analyst', 'analyst_pass', 'log_analyst'),
    ('viewer', 'view_pass', 'readonly_user')
]

user_tokens = {}
for username, password, expected_role in test_users:
    print(f"\n🔐 Authenticating {username}...")
    token = syslog_service.authenticate_user(username, password, expected_role)
    if token:
        user_tokens[username] = token

print(f"\n✅ Authentication complete. Active sessions: {len(user_tokens)}")

🔐 EXERCISE 2: Secure Syslog Management Web Service
✅ Demo environment created with 5 sample log files

🔐 Service Components:
  ✅ Multi-layer authorization system
  ✅ Role-based access control (RBAC)
  ✅ Privilege separation architecture
  ✅ Comprehensive audit logging
  ✅ Secure file operations
  ✅ Input validation and path traversal protection

👥 Available User Roles:
  • syslog_admin: view_audit_logs, delete_old_files, configure_service, list_files, delete_any_file
  • system_admin: list_files, delete_old_files, view_audit_logs
  • log_analyst: list_files, view_audit_logs
  • readonly_user: list_files

DEMONSTRATION: User Authentication & Authorization

🔐 Authenticating admin...
📝 AUDIT: [AUTHENTICATION] admin -> login_success
✅ Authentication successful for admin (syslog_admin)

🔐 Authenticating analyst...
📝 AUDIT: [AUTHENTICATION] analyst -> login_success
✅ Authentication successful for analyst (log_analyst)

🔐 Authenticating viewer...
📝 AUDIT: [AUTHENTICATION] viewer -> login_succ

In [3]:
# Part 2: Demonstrate File Operations with Authorization
print(f"\n" + "="*50)
print("DEMONSTRATION: Secure File Operations")
print("="*50)

# Test 1: List files (available to all authenticated users)
print(f"\n📂 Test 1: List syslog files")
for username, token in user_tokens.items():
    try:
        files = syslog_service.list_syslog_files(token)
        print(f"  ✅ {username}: Found {len(files)} syslog files")
        for file in files[:3]:  # Show first 3
            print(f"    • {file.filename} ({file.age_days} days old, {file.file_type})")
    except Exception as e:
        print(f"  ❌ {username}: {str(e)}")

# Test 2: Attempt file deletion (permission-based)
print(f"\n🗑️  Test 2: Delete old files (DRY RUN)")
for username, token in user_tokens.items():
    try:
        results = syslog_service.delete_old_syslog_files(token, min_age_days=30, dry_run=True)
        print(f"  ✅ {username}: Operation succeeded")
        print(f"    Would delete: {len(results['deleted'])} files")
        print(f"    Would skip: {len(results['skipped'])} files")
    except PermissionError as e:
        print(f"  ❌ {username}: PERMISSION DENIED - {str(e)}")
    except Exception as e:
        print(f"  ❌ {username}: ERROR - {str(e)}")

# Test 3: Actual deletion (admin only)
if 'admin' in user_tokens:
    print(f"\n🗑️  Test 3: Admin performs actual deletion")
    try:
        results = syslog_service.delete_old_syslog_files(
            user_tokens['admin'], 
            min_age_days=40, 
            dry_run=False
        )
        print(f"  ✅ Deletion completed:")
        print(f"    Deleted: {len(results['deleted'])} files")
        print(f"    Skipped: {len(results['skipped'])} files")
        print(f"    Errors: {len(results['errors'])}")
    except Exception as e:
        print(f"  ❌ Admin deletion failed: {str(e)}")

# Test 4: Audit log access
print(f"\n📋 Test 4: Audit Log Access")
for username, token in user_tokens.items():
    try:
        audit_logs = syslog_service.get_audit_logs(token, hours=1)
        print(f"  ✅ {username}: Retrieved {len(audit_logs)} audit events")
    except PermissionError as e:
        print(f"  ❌ {username}: AUDIT ACCESS DENIED - {str(e)}")

# Test 5: Security report (privileged users only)
print(f"\n📊 Test 5: Security Report Generation")
for username, token in user_tokens.items():
    try:
        report = syslog_service.generate_security_report(token)
        print(f"  ✅ {username}: Security report generated")
        print(f"    Total events: {report['total_events']}")
        print(f"    Active sessions: {report['active_sessions']}")
        print(f"    Event categories: {list(report['event_categories'].keys())}")
    except PermissionError as e:
        print(f"  ❌ {username}: REPORT ACCESS DENIED - {str(e)}")

print(f"\n" + "="*50)
print("SECURITY ANALYSIS: Exercise 2 Implementation")
print("="*50)

print(f"""
🔐 AUTHORIZATION LAYERS IMPLEMENTED:

1. AUTHENTICATION LAYER:
   • User credentials validation
   • Secure session token generation
   • Session management with expiration

2. AUTHORIZATION LAYER:
   • Role-based access control (RBAC)
   • Permission-based action validation
   • Session token verification

3. BUSINESS LOGIC LAYER:
   • File age restrictions (minimum 30 days)
   • File type classification (current/archived/rotated)
   • Path traversal protection
   • File size limits

4. PRIVILEGE SEPARATION:
   • Service runs with limited user account
   • Restricted to specific directories
   • Subprocess execution for file operations

5. AUDIT & MONITORING:
   • Comprehensive event logging
   • Authentication tracking
   • Authorization decision recording
   • File operation auditing

🛡️  SECURITY CONTROLS:

✅ Input Validation: File paths, ages, permissions
✅ Path Traversal Protection: Restricted directory access
✅ Least Privilege: Limited service account
✅ Defense in Depth: Multiple authorization layers
✅ Audit Trail: Complete operation logging
✅ Session Management: Token-based with expiration
✅ Error Handling: Secure failure modes

🎯 REAL-WORLD APPLICATIONS:

• Log Management Systems
• File Cleanup Services
• System Administration Tools
• Compliance Reporting Systems
• Audit Trail Management
""")

# Final verification
print(f"\n🔍 Final System State:")
print(f"  • Active sessions: {len(syslog_service.active_sessions)}")
print(f"  • Total audit events: {len(syslog_service.audit_events)}")
print(f"  • Demo files remaining: {len(syslog_service.list_syslog_files(user_tokens['admin']))}")

print(f"\n✅ Exercise 2 Complete: Secure Syslog Management Web Service")
print(f"   This implementation demonstrates proper authorization architecture")
print(f"   with multi-layer security controls and comprehensive audit logging.")


DEMONSTRATION: Secure File Operations

📂 Test 1: List syslog files
📝 AUDIT: [AUTHORIZATION] admin -> access_granted
📝 AUDIT: [FILE_OPERATION] admin -> list_files_success
  ✅ admin: Found 5 syslog files
    • current.log (0 days old, current)
    • app.log.1 (5 days old, rotated)
    • system.log.gz (45 days old, archived)
📝 AUDIT: [AUTHORIZATION] analyst -> access_granted
📝 AUDIT: [FILE_OPERATION] analyst -> list_files_success
  ✅ analyst: Found 5 syslog files
    • current.log (0 days old, current)
    • app.log.1 (5 days old, rotated)
    • system.log.gz (45 days old, archived)
📝 AUDIT: [AUTHORIZATION] viewer -> access_granted
📝 AUDIT: [FILE_OPERATION] viewer -> list_files_success
  ✅ viewer: Found 5 syslog files
    • current.log (0 days old, current)
    • app.log.1 (5 days old, rotated)
    • system.log.gz (45 days old, archived)

🗑️  Test 2: Delete old files (DRY RUN)
📝 AUDIT: [AUTHORIZATION] admin -> access_granted
🗑️  SYSLOG DELETION OPERATION
User: admin
Minimum age: 30 days


## 🎓 Section 2 Summary: Interactive Exercise Solutions

### Key Learning Outcomes

Through these interactive exercises, you have experienced:

1. **Authorization Analysis (Exercise 1)**:
   - Systematic evaluation of authorization mechanisms across different system types
   - Understanding how business requirements drive authorization design
   - Recognition of common patterns and edge cases in access control

2. **Secure Service Implementation (Exercise 2)**:
   - Multi-layer authorization architecture design
   - Implementation of RBAC with proper privilege separation
   - Comprehensive audit logging and security monitoring
   - Real-world security controls in a practical application

### Authorization Best Practices Demonstrated

✅ **Defense in Depth**: Multiple authorization layers working together  
✅ **Least Privilege**: Users and services have minimum necessary permissions  
✅ **Separation of Concerns**: Authentication, authorization, and business logic clearly separated  
✅ **Audit Trail**: Complete logging of all security-relevant decisions and actions  
✅ **Input Validation**: All user inputs validated before processing  
✅ **Secure Defaults**: System fails securely when errors occur  

### Real-World Application

These patterns and techniques are directly applicable to:
- Enterprise web applications
- System administration tools
- API security design
- Compliance and audit systems
- Cloud service authorization

### Next Steps

1. **Practice**: Implement similar authorization systems in your own projects
2. **Study**: Review authorization failures in real-world security incidents
3. **Extend**: Add additional features like dynamic permissions or attribute-based access control (ABAC)
4. **Secure**: Always perform security reviews and penetration testing on authorization systems

---

*Continue to the next chapter to explore advanced security topics and build upon these authorization fundamentals.*